In [ ]:
# 必要なライブラリのインポート
import numpy as np
import pandas as pd
import pickle
import warnings
import unicodedata
import gc
from datetime import timedelta, date
from typing import Dict, List, Optional, Literal

# 外部ライブラリ
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.ticker as plt_ticker
import japanize_matplotlib
import plotly.graph_objects as go
import plotly.express as px

# 社内ライブラリ
from nm.research.supply_chain.py.graph_provider.ciq_region_provider import RegionFromCurrency, RegionGroupForCurrency
from nm.research.supply_chain.panel_regression_model.py.gics_master_provider import GICSMaster
from nm.common.data.data_provider.segment.company_segment_data_provider import CompanySegmentDataProvider
from nm.common.data.database.py.aisg_maria import AISGMaria
from nm.research.supply_chain.common.util.analysis.fiscal_year import FiscalYear
from nm.common.data.database.py.aisg_gib_db import AISGGibDB, GIB_DB
from nm.research.supply_chain.py.data_provider.company_id_map import CompanyIdMap
from nm.research.roic_wacc.py.data_manager_for_ppm import PPMDataManager
from nm.research.roic_wacc.data_provider.ciq_region_provider import RegionFromCurrency, RegionFromIso
from nm.common.data.data_provider.segment.rbics_provider import RBICSDataProvider
from sklearn.preprocessing import minmax_scale


In [ ]:
class DataProcessor:
    """データ処理のための基底クラス"""
    
    def __init__(self):
        self.db = AISGGibDB()
    
    @staticmethod
    def normalize_text(s):
        """テキストの正規化"""
        if isinstance(s, str):
            s_lower = s.lower()
            return unicodedata.normalize('NFKC', s_lower)
        return s
    
    @staticmethod
    def filter_func(group):
        """FSYM_IDが複数ある場合のフィルタリング"""
        if group["FSYM_ID"].nunique() > 1:
            return group[group["PRIMARY_EQUITY_FLAG"] == 1]
        else:
            return group
    
    @staticmethod
    def adjust_fiscal_term(df: pd.DataFrame, date_col: str = "DATE") -> pd.DataFrame:
        """会計期間の調整（3月、6月、9月、12月にマッピング）"""
        _MONTH_MAP = {
            1: 3, 2: 3, 3: 3,
            4: 6, 5: 6, 6: 6,
            7: 9, 8: 9, 9: 9,
            10: 12, 11: 12, 12: 12
        }
        
        df = df.copy()
        df["FTERM"] = pd.to_datetime(df[date_col])
        df["FTERM_2"] = (
            df["FTERM"].dt.year.astype(str) + 
            df["FTERM"].dt.month.map(lambda m: "{:02}".format(_MONTH_MAP[m]))
        ).astype(int)
        df["FISCAL_YEAR"] = df["FTERM_2"].map(FiscalYear.get_fiscal_year)
        
        return df


In [ ]:
class GeographicProcessor:
    """地理的情報の処理クラス"""
    
    def __init__(self):
        self.geographic_regions = [
            "Americas (Excluding US)", "Mideast/Africa", "Africa and Non-US Americas", "Middle East and Africa",
            "Australia/New Zealand", "Other Asia/Pac", "Canada", "Latin America", "Caribbean",
            "Australia and New Zealand", "Other Asia/Pacific", "China", "Africa",
            "Europe", "Eastern Europe", "Northern Europe", "Southern Europe", "Western Europe", "Middle East",
            "Pan-EMEA", "Multinational", "Other United States", "US Territories and Puerto Rico",
            "United States Southern", "United States South Central", "United States Northeast",
            "United States South Atlantic", "United States Atlantic", "United States Pacific",
            "United States West South Central", "United States Western", "Middle Atlantic US",
            "New England United States", "North East United States", "Northwest United States",
            "South United States", "Western United States", "Diversified United States", "United States",
            "Other Americas (Excluding US)", "Australian", "Pan-Asia/Pacific", "Eastern Region",
            "Other US Western", "Pan-US", "US Rockies", "Gulf Coast", "US Mexico", "Mid-Continent",
            "Other US South", "Permian Basin", "Australia/NZ", "North Asia", "North Sea", "Rest of Europe",
            "Pan-Europe", "Other Europe", "MENA", "Russia/CIS/FSU", "Russia/CIS", "Sub-Saharan",
            "Mixed International", "International", "Asia-Europe", "Central and South America", "Mexico",
            "Asia Excluding China", "Americas", "Other Americas", "Global", "Other International",
            "Other Asia/Pacific", "Europe, Middle East and Africa", "United States", "Multinational",
            "Americas", "Asia/Pacific", "US and Canada", "Pan-Americas", "US", "Asia (Excluding China)",
            "Australia and New Zealand", "China", "Pan-Asia/Pacific", "Central and Eastern Europe",
            "Western Europe", "Multi-Region", "Multi-Type United States", "Other United States", "Canada",
            "Latin America", "Other Americas", "Other North America", "Australian", "Pan-Asia", "Rest of Asia",
            "Southeast Asia", "Other Africa", "Russia and CIS", "South Africa", "Central and South America",
            "Pan-Europe", "Middle East/Africa", "Australia including Oceania", "Other Asia/Pacific",
            "Diversified", "Australia/Oceania", "Rest of Asia/Pac", "Europe, Middle East and Africa",
            "United States", "Pan-EMEA", "Asia/Pacific"
        ]
        # 長いものから順にソート（より具体的な地域名を優先）
        self.geographic_regions = sorted(self.geographic_regions, key=len, reverse=True)
    
    def remove_geographic_info(self, text: str) -> str:
        """テキストから地理的情報を除去"""
        for region in self.geographic_regions:
            if text.startswith(region):
                return text[len(region):].lstrip()
        return text
    
    def get_region_mapping(self, currencies: List[str]) -> pd.DataFrame:
        """通貨から地域へのマッピングを取得"""
        region_cls = RegionFromCurrency(custom_group=RegionGroupForCurrency.TYPE_B)
        return pd.DataFrame({
            "REGION": region_cls.get_parent_region(node=currencies),
            "CURRENCY": currencies
        })


In [ ]:
class FinancialDataProcessor(DataProcessor):
    """財務データの取得と処理"""
    
    def __init__(self):
        super().__init__()
        self.geo_processor = GeographicProcessor()
    
    def get_entity_data(self, entity_id: Optional[str] = None) -> pd.DataFrame:
        """企業エンティティデータの取得"""
        sql = f"""
        select
            sym_sec_entity.factset_entity_id as FACTSET_ENTITY_ID,
            sym_ticker_region.fsym_id,
            sym_ticker_region.ticker_region as TICKER_REGION,
            sym_entity.entity_proper_name as FF_CO_NAME,
            sym_entity.iso_country as ISO_COUNTRY_FACT,
            sym_coverage.proper_name as security_proper_name,
            sym_coverage.fref_security_type,
            case
                when sym_coverage.fsym_primary_equity_id = sym_sec_entity.fsym_id then 1
                else 0
            end as PRIMARY_EQUITY_FLAG,
            sym_coverage.fsym_primary_equity_id,
            sym_sec_entity.fsym_id as S_FSYM_ID,
            sym_coverage.active_flag,
            sym_coverage.universe_type
        from
            FACTSET_FEED.sym_v1.sym_sec_entity
        left join
            FACTSET_FEED.sym_v1.sym_coverage
            on sym_sec_entity.fsym_id = sym_coverage.fsym_id
        left join
            FACTSET_FEED.sym_v1.sym_ticker_region
            on sym_coverage.fsym_primary_listing_id = sym_ticker_region.fsym_id
        left join
            FACTSET_FEED.sym_v1.sym_entity
            on sym_entity.factset_entity_id = sym_sec_entity.factset_entity_id
        where
            sym_ticker_region.ticker_region is not null
            {f"and sym_sec_entity.factset_entity_id = '{entity_id}'" if entity_id else ""}
        order by
            sym_sec_entity.factset_entity_id,
            ticker_region
        """
        return self.db.execute_query(sql)
    
    def get_financial_data(self) -> pd.DataFrame:
        """財務データの取得と計算"""
        sql = """
        SELECT BAS.FSYM_ID, BAS.DATE, BAS.CURRENCY, BAS.FF_SALES, BAS.FF_OPER_INC, BAS.FF_ASSETS, BAS_D.FF_NET_INC,
         BAS.FF_ASSETS_CURR, ADV.FF_LIABS_CURR_MISC, BAS.FF_PAY_ACCT, BAS.FF_INC_TAX, BAS.FF_EQ_AFF_INC, BAS_D.FF_EBIT_OPER,
         ADV_D.FF_ROIC FROM FACTSET_FEED.FF_V3.FF_ADVANCED_DER_QF ADV_D
        LEFT JOIN FACTSET_FEED.FF_V3.FF_BASIC_QF BAS ON BAS.FSYM_ID = ADV_D.FSYM_ID AND BAS.DATE = ADV_D.DATE
        LEFT JOIN FACTSET_FEED.FF_V3.FF_ADVANCED_QF ADV ON ADV.FSYM_ID = ADV_D.FSYM_ID AND ADV.DATE = ADV_D.DATE
        LEFT JOIN FACTSET_FEED.FF_V3.FF_BASIC_DER_QF BAS_D ON BAS_D.FSYM_ID = ADV_D.FSYM_ID AND BAS_D.DATE = ADV_D.DATE
        WHERE BAS.DATE >= '2015-4-28' AND BAS.FF_SALES != 0
        """
        
        df = self.db.execute_query(sql)
        return self._calculate_financial_metrics(df)
    
    def _calculate_financial_metrics(self, df: pd.DataFrame) -> pd.DataFrame:
        """財務指標の計算"""
        df = df.copy()
        
        # 基本計算
        df["FF_ROIC"] /= 100
        df["固定資産合計"] = df["FF_ASSETS"] - df["FF_ASSETS_CURR"]
        df["投下資本(運用ベース)"] = (
            df["固定資産合計"] + 
            df["FF_ASSETS_CURR"] - 
            df["FF_PAY_ACCT"].fillna(0) - 
            df["FF_LIABS_CURR_MISC"].fillna(0)
        )
        df["税引後営業利益"] = (
            df["FF_OPER_INC"] - 
            df["FF_INC_TAX"].fillna(0) + 
            df["FF_EQ_AFF_INC"].fillna(0)
        )
        
        # 会計期間の調整
        df = self.adjust_fiscal_term(df)
        
        # 2024年12月未満のデータのみ
        df = df.query("FTERM_2 < 202412")
        
        # 地域マッピング
        currencies = df["CURRENCY"].unique()
        region_df = self.geo_processor.get_region_mapping(currencies)
        df = df.merge(region_df, on=["CURRENCY"], how="left")
        
        return df


In [ ]:
class SegmentDataProcessor(DataProcessor):
    """セグメントデータの処理"""
    
    def get_segment_data(self) -> pd.DataFrame:
        """セグメントデータの取得"""
        sql = """
        SELECT FSYM_ID, DATE, FF_SEGMENT_NUM, CURRENCY, LABEL, SALES AS SALES_ER, OPINC, ASSETS, INTERSEG_REV AS SALES_IR
        FROM FACTSET_FEED.FF_V3.FF_SEGBUS_AF SEG
        WHERE SEG.DATE >= '2015-4-28'
        """
        
        df = self.db.execute_query(sql)
        return self._process_segment_data(df)
    
    def _process_segment_data(self, df: pd.DataFrame) -> pd.DataFrame:
        """セグメントデータの処理"""
        df = df.copy()
        df["LABEL_2"] = df["LABEL"].apply(self.normalize_text)
        
        # 会計期間の調整
        df = self.adjust_fiscal_term(df)
        
        # 売上の計算
        df["SALES"] = df["SALES_ER"] + df["SALES_IR"].fillna(0)
        
        # 売上高が正の企業のみ
        df = df.query("SALES > 0")
        
        # Reconciling Itemsの処理
        recon_df = df.query("LABEL == 'Reconciling Items'").reset_index(drop=True)
        recon_df = recon_df[["FSYM_ID", "FISCAL_YEAR", "SALES"]].drop_duplicates()
        recon_df = recon_df.rename(columns={"SALES": "RECON_SALES"})
        
        df = df.query("LABEL != 'Reconciling Items'").reset_index(drop=True)
        
        # 売上高・営業利益・資産の合計計算
        sum_cols = ["SALES", "SALES_ER", "OPINC", "ASSETS"]
        sum_df = (df.groupby(["FSYM_ID", "FTERM_2"])[sum_cols].sum()
                   .reset_index()
                   .rename(columns={col: f"{col}_SUM" for col in sum_cols}))
        
        df = df.merge(sum_df, on=["FSYM_ID", "FTERM_2"])
        df = df.merge(recon_df, how="left", on=["FSYM_ID", "FISCAL_YEAR"])
        
        # 比率の計算
        df = self._calculate_ratios(df)
        
        return df
    
    def _calculate_ratios(self, df: pd.DataFrame) -> pd.DataFrame:
        """各種比率の計算"""
        # ゼロを NaN に置換
        for col in ["SALES_SUM", "SALES_ER_SUM", "OPINC_SUM", "ASSETS_SUM"]:
            df[col] = df[col].replace(0, np.nan)
        
        # 比率計算
        df["SALES_RATIO"] = df["SALES"] / (df["SALES_ER_SUM"] + df["RECON_SALES"].fillna(0))
        df["SALES_ER_RATIO"] = df["SALES_ER"] / df["SALES_ER_SUM"]
        df["OPINC_RATIO"] = df["OPINC"] / df["OPINC_SUM"]
        df["ASSETS_RATIO"] = df["ASSETS"] / df["ASSETS_SUM"]
        
        # 必要な列のみ選択（entity情報はここではまだ含まれていない）
        columns = [
            "FSYM_ID", "LABEL", "LABEL_2", "FTERM_2", "FISCAL_YEAR",
            "SALES", "SALES_ER", "SALES_IR", "OPINC", "ASSETS",
            "OPINC_RATIO", "ASSETS_RATIO", "SALES_RATIO", "SALES_ER_RATIO", "RECON_SALES"
        ]
        
        return df[columns]


In [ ]:
class RBICSProcessor:
    """RBICSデータの処理"""
    
    def __init__(self):
        self.rbics_provider = RBICSDataProvider()
        self.db = AISGGibDB()
    
    def get_master_data(self) -> pd.DataFrame:
        """RBICSマスターデータの取得"""
        return self.rbics_provider.get_master_table()
    
    def get_rename_data(self) -> pd.DataFrame:
        """RBICS リネームデータの取得"""
        query = """
        SELECT RBICS as L6_ID_ADJ, NAME as NAME_ADJ, L6_ID, L6_NAME FROM WORK_TEMP.dbo.YS_RBICS
        WHERE CREATE_DATE = (SELECT MAX(CREATE_DATE) FROM WORK_TEMP.dbo.YS_RBICS)
        """
        df = self.db.execute_query(query)
        df['MAX_LEVEL'] = df['L6_ID_ADJ'].apply(lambda x: len(str(x)) // 2)
        df["MAX_LEVEL_NAME"] = df["NAME_ADJ"]
        return df


In [ ]:
class RevereDataProcessor(DataProcessor):
    """REVEREデータの処理"""
    
    def get_revere_data(self) -> pd.DataFrame:
        """REVEREセグメントデータの取得"""
        sql = """
        SELECT DISTINCT
            F.COMPANY_ID,
            G.FS_ENTITY_ID AS FACTSET_ENTITY_ID,
            D.NAME AS COMPANY_NAME,
            D.SEDOL, D.TICKER, D.ISIN, D.CUSIP,
            H.NAME AS REGION_NAME,
            H.DOME_REGION AS REGION_CODE,
            H.COUNTRY AS HQ_REGION_CODE,
            A.PERIOD_END_DATE,
            B.ID   AS SEGMENT_ID,
            B.NAME AS SEGMENT_NAME,
            B.TYPE,
            B.REVENUE_PERCENT AS SEG_SHARE,
            B.RBICS2_L6_ID AS REVENUE_L6_ID
        FROM FACTSET_REVERE.COMPANY_RBICS2_BUS_SEG_REPORT A
        INNER JOIN (
            SELECT
                COMPANY_ID,
                MAX(PERIOD_END_DATE) AS PERIOD_END_DATE,
                MAX(STARTS) AS STARTS
            FROM FACTSET_REVERE.COMPANY_RBICS2_BUS_SEG_REPORT
            GROUP BY COMPANY_ID
        ) F ON A.COMPANY_ID = F.COMPANY_ID
        INNER JOIN FACTSET_REVERE.COMPANY_RBICS2_BUS_SEG_ITEM B
            ON A.ID = B.REPORT_ID
        INNER JOIN FACTSET_REVERE.COMPANY_FACTSET G
            ON A.COMPANY_ID = G.COMPANY_ID
        WHERE A.PERIOD_END_DATE >= CONVERT(DATETIME, '2017-04-01 00:00:00')
          AND B.TYPE != 'N'
        """
        
        df = self.db.execute_query(sql)
        return self._process_revere_data(df)
    
    def _process_revere_data(self, df: pd.DataFrame) -> pd.DataFrame:
        """REVEREデータの処理"""
        df = df.copy()
        
        # セグメント名の正規化
        df["SEGMENT_NAME"] = df["SEGMENT_NAME"].str.strip()
        df["SEGMENT_NAME"] = df["SEGMENT_NAME"].str.replace(r"[—–]", "-", regex=True)
        df["SEGMENT_NAME"] = df["SEGMENT_NAME"].str.replace(r"(\\w)\\s", r"\\1 ", regex=True)
        
        # 売上比率と会計年度の計算
        df["SALES_RATIO"] = df["SEG_SHARE"].astype(float) / 100
        df["FISCAL_YEAR"] = df["PERIOD_END_DATE"].map(FiscalYear.get_fiscal_year)
        
        # セグメントシェアの合計計算
        df["SEG_SHARE_SUM"] = (
            df.groupby(["FACTSET_ENTITY_ID", "FISCAL_YEAR"])["SEG_SHARE"]
            .transform("sum")
        )
        
        df.sort_values(
            by=["FACTSET_ENTITY_ID", "FISCAL_YEAR", "SALES_RATIO"],
            inplace=True
        )
        
        # 冗長な行の除去とセグメント名の処理
        df = df.groupby(['FACTSET_ENTITY_ID', 'FISCAL_YEAR'], group_keys=False).apply(
            self._remove_redundant_rows
        )
        
        return df
    
    def _remove_redundant_rows(self, group):
        """重複するセグメント行の除去"""
        flag = group["SEG_SHARE"].sum() != 1
        indices_to_remove = []
        
        for idx, seg in group["SEGMENT_NAME"].items():
            search_str = seg + " "
            if not flag:
                if any(other_seg.startswith(search_str) for j, other_seg in group["SEGMENT_NAME"].items() if j != idx):
                    indices_to_remove.append(idx)
            
            for j, other_seg in group["SEGMENT_NAME"].items():
                if other_seg.startswith(search_str):
                    if not other_seg.startswith(seg + " - "):
                        updated_seg = seg + " - " + other_seg[len(seg):].lstrip()
                        group.at[j, "SEGMENT_NAME"] = updated_seg
        
        return group.drop(indices_to_remove)


In [ ]:
class FactSetDataManager:
    """FactSetデータの統合処理クラス"""
    
    def __init__(self):
        self.financial_processor = FinancialDataProcessor()
        self.segment_processor = SegmentDataProcessor()
        self.revere_processor = RevereDataProcessor()
        self.rbics_processor = RBICSProcessor()
        self.geo_processor = GeographicProcessor()
    
    def initialize_data(self) -> Dict[str, pd.DataFrame]:
        """初期データの取得・処理"""
        print("データの取得を開始...")
        
        # 基本データの取得
        entity_data = self.financial_processor.get_entity_data()
        financial_data = self.financial_processor.get_financial_data()
        segment_data = self.segment_processor.get_segment_data()
        revere_data = self.revere_processor.get_revere_data()
        rbics_master = self.rbics_processor.get_master_data()
        
        print(f"企業データ: {len(entity_data)} 件")
        print(f"財務データ: {len(financial_data)} 件")
        print(f"セグメントデータ: {len(segment_data)} 件")
        print(f"REVEREデータ: {len(revere_data)} 件")
        
        # 企業データのマージ
        financial_data = financial_data.merge(
            entity_data[["FSYM_ID", "FACTSET_ENTITY_ID", "FF_CO_NAME", "PRIMARY_EQUITY_FLAG"]],
            on="FSYM_ID"
        )
        
        segment_data = segment_data.merge(
            entity_data[["FSYM_ID", "FACTSET_ENTITY_ID", "FF_CO_NAME", "PRIMARY_EQUITY_FLAG"]],
            on="FSYM_ID"
        )
        
        # プライマリーエクイティのフィルタリング
        financial_data = financial_data.groupby('FACTSET_ENTITY_ID', group_keys=False).apply(
            DataProcessor.filter_func
        )
        segment_data = segment_data.groupby('FACTSET_ENTITY_ID', group_keys=False).apply(
            DataProcessor.filter_func
        )
        
        # seg_af2とseg_af3の作成（オリジナルコードと同等）
        segment_data = self._create_segment_variations(segment_data)
        
        return {
            "entity": entity_data,
            "financial": financial_data,
            "segment": segment_data,
            "revere": revere_data,
            "rbics_master": rbics_master
        }
    
    def _create_segment_variations(self, segment_data: pd.DataFrame) -> pd.DataFrame:
        """seg_af2とseg_af3を作成（オリジナルコードのseg_af2, seg_af3と同等）"""
        columns_base = [
            "FSYM_ID", "FACTSET_ENTITY_ID", "LABEL", "LABEL_2", "FTERM_2", "FISCAL_YEAR",
            "SALES", "SALES_ER", "SALES_IR", "OPINC", "ASSETS",
            "OPINC_RATIO", "ASSETS_RATIO", "RECON_SALES", "PRIMARY_EQUITY_FLAG"
        ]
        
        # seg_af2相当（SALES_RATIOを使用）
        seg_af2 = segment_data[columns_base + ["SALES_RATIO"]].copy()
        
        # seg_af3相当（SALES_ER_RATIOをSALS_RATIOとしてリネーム）
        seg_af3 = segment_data[columns_base + ["SALES_ER_RATIO"]].copy()
        seg_af3 = seg_af3.rename(columns={"SALES_ER_RATIO": "SALES_RATIO"})
        
        # 連結して重複除去
        result = pd.concat([seg_af2, seg_af3], axis=0).reset_index(drop=True)
        result.drop_duplicates(inplace=True)
        
        return result
    
    def merge_segment_mapping(self, data: Dict[str, pd.DataFrame]) -> pd.DataFrame:
        """セグメントマッピングの統合"""
        revere_data = data["revere"]
        segment_data = data["segment"]
        entity_data = data["entity"]
        
        # セグメント名の正規化
        revere_processed = self._process_revere_segments(revere_data)
        
        # マッピングの実行
        merged = self._perform_segment_mapping(revere_processed, segment_data)
        
        # 企業情報の追加
        merged = merged.merge(
            entity_data[['FACTSET_ENTITY_ID', 'ISO_COUNTRY_FACT', 'FF_CO_NAME']].drop_duplicates(),
            on=['FACTSET_ENTITY_ID']
        )
        
        # ** 重要：セグメントデータの財務指標をマージ **
        # オリジナルのseg_af_mergedと同等の処理
        # pivot処理でprocess_entity関数と同等の処理を実行
        import warnings
        def process_entity(group):
            group_filled = group.fillna("-9999")
            pivot = group_filled.pivot_table(
                index=["FACTSET_ENTITY_ID", "FISCAL_YEAR"], 
                columns="SEGMENT_NAME_3",
                values=["LABEL", "REVENUE_L6_ID"], 
                aggfunc="first"
            )
            pivot_filled = pivot.copy()
            for col in pivot.columns:
                with warnings.catch_warnings():
                    warnings.simplefilter("ignore", category=FutureWarning)
                    orig_nan = pivot[col].isna()
                    tmp = pivot[col].replace("-9999", np.nan)
                    filled = tmp.bfill().ffill()
                    filled[orig_nan] = np.nan
                    pivot_filled[col] = filled

            stacked = pivot_filled.stack(level=1, future_stack=True).reset_index()
            stacked["FACTSET_ENTITY_ID"] = group["FACTSET_ENTITY_ID"].iloc[0]
            return stacked

        result = merged.groupby("FACTSET_ENTITY_ID", group_keys=False).apply(process_entity)
        
        # セグメントデータとマージして財務指標を追加
        seg_af_merged = segment_data.merge(
            result.query("LABEL.notna()"),
            left_on=["FACTSET_ENTITY_ID", "FISCAL_YEAR", "LABEL"],
            right_on=["FACTSET_ENTITY_ID", "FISCAL_YEAR", "LABEL"],
            how="inner"
        )
        
        return seg_af_merged
    
    def _process_revere_segments(self, revere_data: pd.DataFrame) -> pd.DataFrame:
        """REVEREセグメントの正規化処理"""
        df = revere_data.copy()
        
        # セグメント名の正規化処理
        df['SEGMENT_NAME_2'] = df['SEGMENT_NAME']
        df['SEGMENT_NAME_3'] = df['SEGMENT_NAME_2'].str.split(' - ').str[0].str.strip()
        
        # 不要なキーワードの除去
        target_blocks = ['Business', 'Sector']
        pattern = r'\\s*(?:' + '|'.join(target_blocks) + r')$'
        df['SEGMENT_NAME_3'] = df['SEGMENT_NAME_3'].str.replace(pattern, '', regex=True).str.strip()
        df['SEGMENT_NAME_3'] = df['SEGMENT_NAME_3'].apply(DataProcessor.normalize_text)
        
        return df
    
    def _perform_segment_mapping(self, revere_data: pd.DataFrame, segment_data: pd.DataFrame) -> pd.DataFrame:
        """セグメントマッピングの実行"""
        # merge_asofを使用したマッピング
        merged = pd.merge_asof(
            revere_data[['FACTSET_ENTITY_ID', 'SEGMENT_NAME', 'FISCAL_YEAR', 'REVENUE_L6_ID', 'SALES_RATIO', 'SEGMENT_NAME_3']]
                .drop_duplicates()
                .sort_values(by=['SALES_RATIO']),
            segment_data.dropna(subset=['SALES_RATIO'])
                .drop_duplicates(subset=['FACTSET_ENTITY_ID', 'FTERM_2', 'LABEL', 'SALES_RATIO'])
                [['FACTSET_ENTITY_ID', 'LABEL', 'FISCAL_YEAR', 'SALES_RATIO']]
                .sort_values(by=['SALES_RATIO']),
            by=['FACTSET_ENTITY_ID', 'FISCAL_YEAR'],
            on='SALES_RATIO',
            direction='nearest',
            tolerance=0.001
        ).drop_duplicates()
        
        # 文字列一致による追加マッピング
        merged2 = merged.merge(
            segment_data[['FACTSET_ENTITY_ID', 'FISCAL_YEAR', 'LABEL', 'LABEL_2']],
            left_on=['FACTSET_ENTITY_ID', 'FISCAL_YEAR', 'SEGMENT_NAME_3'],
            right_on=['FACTSET_ENTITY_ID', 'FISCAL_YEAR', 'LABEL_2'],
            how='left',
            suffixes=('', '_new')
        )
        
        merged2['LABEL'] = merged2['LABEL'].fillna(merged2['LABEL_new'])
        merged2.drop(columns=['LABEL_new', 'LABEL_2'], inplace=True)
        merged2.drop_duplicates(inplace=True)
        
        return merged2


In [ ]:
class ProductScoreCalculator:
    """製品スコア計算クラス"""
    
    def __init__(self, geo_processor: GeographicProcessor):
        self.geo_processor = geo_processor
    
    def calculate_segment_scores(self, financial_data: pd.DataFrame, segment_mapping: pd.DataFrame, 
                               rbics_master: pd.DataFrame, mapping_df: pd.DataFrame = None) -> pd.DataFrame:
        """セグメント別スコアの計算"""
        
        # セグメント・財務データの統合
        merged_data = financial_data.merge(
            segment_mapping[["FACTSET_ENTITY_ID", "LABEL", "FISCAL_YEAR", 
                           "SALES", "SALES_ER", "OPINC", "ASSETS",
                           "OPINC_RATIO", "ASSETS_RATIO", "SALES_RATIO", "REVENUE_L6_ID"]],
            on=["FACTSET_ENTITY_ID", "FISCAL_YEAR"],
            how="inner"
        )
        

        # 売上比率の平均計算
        merged_data["SALES_RATIO"] = (
            merged_data.groupby(['FSYM_ID', 'FTERM_2', 'LABEL'])["SALES_RATIO"]
            .transform("mean")
        )
        merged_data.drop_duplicates(inplace=True)
        
        # セグメント別財務指標の計算
        merged_data["SEG_税引後営業利益"] = merged_data["税引後営業利益"] * merged_data["OPINC_RATIO"]
        merged_data["SEG_投下資本(運用ベース)"] = merged_data["投下資本(運用ベース)"] * merged_data["ASSETS_RATIO"]
        
        # 必要な列の選択
        result_columns = [
            "FSYM_ID", "FACTSET_ENTITY_ID", "FF_CO_NAME", "CURRENCY",
            "FTERM_2", "FISCAL_YEAR", "LABEL",
            "固定資産合計", "投下資本(運用ベース)",
            "SEG_税引後営業利益", "SEG_投下資本(運用ベース)",
            "SALES_ER", "SALES_RATIO", "REVENUE_L6_ID"
        ]
        
        merged_data = merged_data[result_columns]
        
        # REVENUE_L6_IDの展開
        merged_data["REVENUE_L6_ID"] = (
            merged_data["REVENUE_L6_ID"]
            .str.split(',')
            .apply(lambda ids: [s.strip() for s in ids] if isinstance(ids, list) else [])
        )
        merged_data = merged_data.explode("REVENUE_L6_ID")
        
        # マッピングデータの適用
        if mapping_df is not None:
            identity_mapping = merged_data[["FACTSET_ENTITY_ID", "REVENUE_L6_ID", "REVENUE_L6_ID"]].drop_duplicates()
            identity_mapping.columns = ["FACTSET_ENTITY_ID", "PRODUCT_L6_ID", "RELABEL_L6_ID"]
            
            combined_mapping = pd.concat([mapping_df, identity_mapping], axis=0).drop_duplicates()
            combined_mapping.reset_index(drop=True, inplace=True)
            
            # PRODUCT_L6_NAMEを作成するためのマージ（オリジナルコードと同じ処理）
            combined_mapping = combined_mapping.merge(
                rbics_master[["L6_ID", "L6_NAME"]].rename(
                    columns={"L6_ID": "PRODUCT_L6_ID", "L6_NAME": "PRODUCT_L6_NAME"}
                ),
                how="left",
                on="PRODUCT_L6_ID"
            )
            
            merged_data = merged_data.merge(
                combined_mapping.rename(columns={"RELABEL_L6_ID": "REVENUE_L6_ID"}),
                left_on=["FACTSET_ENTITY_ID", "REVENUE_L6_ID"],
                right_on=["FACTSET_ENTITY_ID", "REVENUE_L6_ID"],
                how="left"
            )
        
        # 依存度の計算
        merged_data = self._calculate_dependencies(merged_data)
        
        return merged_data
    
    def _calculate_dependencies(self, df: pd.DataFrame) -> pd.DataFrame:
        """各種依存度の計算"""
        df = df.copy()
        
        # 売上比率依存度の計算
        df["SALES_RATIO_DEPENDENCY"] = (
            df.groupby(["FACTSET_ENTITY_ID", "FTERM_2", "LABEL"])["SALES_RATIO"]
            .transform(lambda x: x / x.sum())
        )
        
        # セグメント調整コードの作成
        df["SEGMENT_ADJ"] = df["LABEL"].str.strip().str.replace(" ", "_")
        df["CODE_SEGMENT_ADJ"] = df["FACTSET_ENTITY_ID"] + "_" + df["SEGMENT_ADJ"]
        
        # セグメントシェアの計算
        df["SEG_SHARE"] = df["SALES_RATIO_DEPENDENCY"] * df["SALES_RATIO"]
        
        # セグメント正規化依存度の計算
        df["セグメント正規化依存度"] = (
            df.groupby(["FACTSET_ENTITY_ID", "SEGMENT_ADJ", "FTERM_2"])["SEG_SHARE"]
            .transform(lambda x: x / x.sum())
        )
        
        # 企業全体の正規化依存度の計算
        df["正規化依存度"] = (
            df.groupby(["FACTSET_ENTITY_ID", "FTERM_2"])["SALES_RATIO"]
            .transform(lambda x: x / x.sum())
        )
        
        return df
    
    def create_pivot_tables(self, df: pd.DataFrame, rbics_master: pd.DataFrame, 
                          currency_filter: Optional[str] = 'USD') -> Dict[str, pd.DataFrame]:
        """ピボットテーブルの作成"""
        
        # 通貨フィルタ（オプショナル）
        if currency_filter is not None:
            filtered_df = df.query(f"CURRENCY == '{currency_filter}'")
        else:
            filtered_df = df
        
        # L6レベルのピボットテーブル（必ずPRODUCT_L6_NAMEを使用）
        product_name_col = 'PRODUCT_L6_NAME'
        
        # セグメント別製品スコア
        segment_product_score = pd.pivot_table(
            filtered_df, 
            index=['CODE_SEGMENT_ADJ', 'FTERM_2'], 
            values='セグメント正規化依存度',
            columns=[product_name_col], 
            fill_value=0
        )
        
        # 連結企業製品スコア
        consol_product_score = pd.pivot_table(
            filtered_df, 
            index=['FACTSET_ENTITY_ID', 'FTERM_2'], 
            values='正規化依存度',
            columns=[product_name_col], 
            fill_value=0, 
            aggfunc='sum'
        )
        
        # 階層レベルでの集約
        level_mappings = self._create_level_mappings(rbics_master, product_name_col)
        
        result = {
            'segment_l6': segment_product_score,
            'consol_l6': consol_product_score
        }
        
        # L5, L4レベルの集約
        for level in ['L5', 'L4']:
            if level in level_mappings:
                mapping = level_mappings[level]
                
                result[f'segment_{level.lower()}'] = (
                    segment_product_score.rename(columns=mapping)
                    .groupby(level=0, axis=1).sum()
                )
                
                result[f'consol_{level.lower()}'] = (
                    consol_product_score.rename(columns=mapping)
                    .groupby(level=0, axis=1).sum()
                )
        
        return result
    
    def _create_level_mappings(self, rbics_master: pd.DataFrame, 
                             product_name_col: str) -> Dict[str, Dict]:
        """階層レベルマッピングの作成"""
        mappings = {}
        
        if product_name_col == 'PRODUCT_L6_NAME':
            # L6からL5へのマッピング
            mappings['L5'] = (
                rbics_master.drop_duplicates(subset=["L6_NAME"])
                .set_index('L6_NAME')["L5_NAME"]
                .to_dict()
            )
            
            # L6からL4へのマッピング
            mappings['L4'] = (
                rbics_master.drop_duplicates(subset=["L6_NAME"])
                .set_index('L6_NAME')["L4_NAME"]
                .to_dict()
            )
        
        return mappings
    
    def remove_geographic_info(self, pivot_tables: Dict[str, pd.DataFrame]) -> Dict[str, pd.DataFrame]:
        """地理的情報の除去"""
        processed_tables = {}
        
        for name, df in pivot_tables.items():
            print(f"{name}: {len(df.columns)} 列")
            
            # 地理的情報の除去
            column_mapping = {col: self.geo_processor.remove_geographic_info(col) for col in df.columns}
            processed_df = df.rename(columns=column_mapping).groupby(level=0, axis=1).sum()
            
            print(f"{name} (処理後): {len(processed_df.columns)} 列")
            processed_tables[name] = processed_df
        
        return processed_tables
    
    def save_pivot_tables(self, pivot_tables: Dict[str, pd.DataFrame], 
                         output_dir: str = "/home/tmiyahara/repos/Neumann-Notebook/tmiyahara/202505/"):
        """ピボットテーブルの保存"""
        for name, df in pivot_tables.items():
            # ファイル保存
            filename = f"{output_dir}{name}_product_share.pkl"
            df.to_pickle(filename)
            print(f"保存完了: {filename}")
            
            # メモリ解放
            del df
            gc.collect()


In [ ]:
# メイン実行関数
def main():
    """メイン実行関数"""
    print("=== FactSet データ処理開始 ===")
    
    # 1. データマネージャーの初期化
    manager = FactSetDataManager()
    
    # 2. 初期データの取得
    data = manager.initialize_data()
    
    # 3. セグメントマッピングの実行
    print("\\nセグメントマッピングを実行中...")
    segment_mapping = manager.merge_segment_mapping(data)
    
    # 4. 製品スコア計算の実行
    print("\\n製品スコアを計算中...")
    calculator = ProductScoreCalculator(manager.geo_processor)
    
    # マッピングデータの読み込み（必要に応じて）
    mapping_df = None
    try:
        with open("/home/tmiyahara/repos/Neumann-Notebook/tmiyahara/202501/mapping_df.pkl", "rb") as f:
            mapping_df = pickle.load(f)
            mapping_df = mapping_df[["FACTSET_ENTITY_ID", "PRODUCT_L6_ID", "RELABEL_L6_ID"]]
            print(f"マッピングデータを読み込みました: {len(mapping_df)} 件")
    except FileNotFoundError:
        print("マッピングデータが見つかりません。デフォルト処理を実行します。")
    
    # セグメントスコアの計算
    segment_scores = calculator.calculate_segment_scores(
        data["financial"], 
        segment_mapping, 
        data["rbics_master"],
        mapping_df
    )
    
    # 5. ピボットテーブルの作成
    print("\\nピボットテーブルを作成中...")
    pivot_tables = calculator.create_pivot_tables(
        segment_scores, 
        data["rbics_master"], 
        currency_filter='USD'
    )
    
    # 6. 地理的情報の除去
    print("\\n地理的情報を除去中...")
    processed_pivot_tables = calculator.remove_geographic_info(pivot_tables)
    
    # 7. 結果の保存
    print("\\n結果を保存中...")
    calculator.save_pivot_tables(processed_pivot_tables)
    
    print("\\n=== 処理完了 ===")
    
    return {
        "data": data,
        "segment_mapping": segment_mapping,
        "segment_scores": segment_scores,
        "pivot_tables": pivot_tables,
        "processed_pivot_tables": processed_pivot_tables
    }

# 実行
if __name__ == "__main__":
    results = main()


In [ ]:
# 個別実行のサンプル

# 特定企業のデータのみを取得したい場合
def get_specific_company_data(entity_id: Optional[str] = None):
    """特定企業のデータを取得（Noneの場合は全企業）"""
    financial_processor = FinancialDataProcessor()
    entity_data = financial_processor.get_entity_data(entity_id)
    return entity_data

# 地理的情報の除去のみを実行したい場合
def clean_geographic_data(text_list: List[str]):
    """地理的情報の除去"""
    geo_processor = GeographicProcessor()
    return [geo_processor.remove_geographic_info(text) for text in text_list]

# RBICSマスターデータのみを取得したい場合
def get_rbics_master():
    """RBICSマスターデータの取得"""
    rbics_processor = RBICSProcessor()
    return rbics_processor.get_master_data()

print("個別実行のサンプルが定義されました。必要に応じてコメントアウトを外して実行してください。")


In [ ]:
# 詳細分析用のユーティリティ関数（最新期間対応版）
def explore_specific_category(category_companies: Dict[str, pd.DataFrame], 
                            category_name: str, 
                            top_n: int = 20) -> pd.DataFrame:
    """
    特定カテゴリーの詳細探索（最新期間のみ）
    
    Args:
        category_companies: カテゴリー別企業辞書
        category_name: 分析対象カテゴリー名
        top_n: 表示する企業数
        
    Returns:
        詳細データフレーム
    """
    if category_name not in category_companies:
        print(f"カテゴリー '{category_name}' が見つかりません。")
        print("利用可能なカテゴリー:")
        for i, cat in enumerate(sorted(category_companies.keys())[:10]):
            print(f"  {i+1}. {cat}")
        return pd.DataFrame()
    
    df = category_companies[category_name].head(top_n)
    
    print(f"=== {category_name} カテゴリー詳細 ===")
    print(f"該当企業数: {len(category_companies[category_name])}")
    print(f"表示企業数: {min(top_n, len(df))}")
    
    if 'FF_SALES' in df.columns:
        print(f"総売上高: {df['FF_SALES'].sum():,.0f}")
        print(f"平均売上高: {df['FF_SALES'].mean():,.0f}")
        print(f"平均依存度: {df['CATEGORY_VALUE'].mean():.3f}")
    
    print(f"\n上位企業:")
    display_cols = ['FF_CO_NAME', 'CATEGORY_VALUE', 'FF_SALES', 'CURRENCY', 'FTERM_2']
    available_cols = [col for col in display_cols if col in df.columns]
    print(df[available_cols].to_string(index=False))
    
    return df

def compare_categories(summary_df: pd.DataFrame, 
                      category_companies: Dict[str, pd.DataFrame],
                      categories: List[str]) -> go.Figure:
    """
    複数カテゴリーの比較可視化
    
    Args:
        summary_df: カテゴリーサマリー
        category_companies: カテゴリー別企業辞書
        categories: 比較対象カテゴリーリスト
        
    Returns:
        比較グラフ
    """
    fig = make_subplots(
        rows=2, cols=2,
        subplot_titles=('企業数', '総売上高', '平均依存度', '最大依存度'),
        specs=[[{'type': 'bar'}, {'type': 'bar'}],
               [{'type': 'bar'}, {'type': 'bar'}]]
    )
    
    # フィルタリング
    filtered_summary = summary_df[summary_df['Category'].isin(categories)]
    
    if len(filtered_summary) == 0:
        print("指定されたカテゴリーが見つかりません")
        return None
    
    # グラフの作成
    fig.add_trace(
        go.Bar(x=filtered_summary['Category'], y=filtered_summary['Company_Count'], name='企業数'),
        row=1, col=1
    )
    
    if 'Total_Sales' in filtered_summary.columns:
        fig.add_trace(
            go.Bar(x=filtered_summary['Category'], y=filtered_summary['Total_Sales'], name='総売上高'),
            row=1, col=2
        )
    
    fig.add_trace(
        go.Bar(x=filtered_summary['Category'], y=filtered_summary['Avg_Dependency'], name='平均依存度'),
        row=2, col=1
    )
    
    fig.add_trace(
        go.Bar(x=filtered_summary['Category'], y=filtered_summary['Max_Dependency'], name='最大依存度'),
        row=2, col=2
    )
    
    fig.update_layout(
        title='カテゴリー比較',
        height=800,
        showlegend=False
    )
    
    return fig

def get_cross_category_companies(category_companies: Dict[str, pd.DataFrame],
                               categories: List[str],
                               min_dependency: float = 0.05) -> pd.DataFrame:
    """
    複数カテゴリーにまたがる企業の分析
    
    Args:
        category_companies: カテゴリー別企業辞書
        categories: 対象カテゴリーリスト
        min_dependency: 最小依存度閾値
        
    Returns:
        クロスカテゴリー企業データフレーム
    """
    cross_companies = []
    
    for category in categories:
        if category in category_companies:
            df = category_companies[category]
            df = df[df['CATEGORY_VALUE'] >= min_dependency]
            df['CATEGORY'] = category
            cross_companies.append(df[['FACTSET_ENTITY_ID', 'FF_CO_NAME', 
                                     'CATEGORY_VALUE', 'CATEGORY', 'FF_SALES']])
    
    if not cross_companies:
        return pd.DataFrame()
    
    combined = pd.concat(cross_companies, ignore_index=True)
    
    # 複数カテゴリーに属する企業の抽出
    entity_counts = combined['FACTSET_ENTITY_ID'].value_counts()
    multi_category_entities = entity_counts[entity_counts > 1].index
    
    result = combined[combined['FACTSET_ENTITY_ID'].isin(multi_category_entities)]
    result = result.sort_values(['FACTSET_ENTITY_ID', 'CATEGORY_VALUE'], ascending=[True, False])
    
    print(f"複数カテゴリーに属する企業数: {len(multi_category_entities)}")
    print(f"対象カテゴリー: {', '.join(categories)}")
    
    return result

print("詳細分析用関数が定義されました。")


In [ ]:
# 可視化用ライブラリの追加インポート
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.offline as pyo

# インタラクティブな表示のため
pyo.init_notebook_mode(connected=True)

class CategoryAnalyzer:
    """カテゴリー別企業分析クラス（最新期間対応版）"""
    
    def __init__(self, pivot_tables_dict: Dict[str, pd.DataFrame], 
                 financial_data: pd.DataFrame = None):
        self.pivot_tables = pivot_tables_dict
        self.financial_data = financial_data
        
    def analyze_category_companies(self, table_name: str = 'consol_l5', 
                                 threshold: float = 0.01, 
                                 specific_period: Optional[int] = None) -> Dict[str, pd.DataFrame]:
        """
        各カテゴリーに属する企業の分析
        
        Args:
            table_name: 分析対象のテーブル名
            threshold: カテゴリーに属すると判断する最小閾値
            specific_period: 特定の期間を指定（例：202403）。Noneの場合は最新期間を使用
            
        Returns:
            各カテゴリーの企業リスト辞書
        """
        if table_name not in self.pivot_tables:
            raise ValueError(f"Table {table_name} not found in pivot_tables")
        
        df = self.pivot_tables[table_name]
        
        # 期間の制限処理
        if specific_period is not None:
            # 特定期間が指定された場合
            available_periods = df.index.get_level_values(1).unique()
            if specific_period not in available_periods:
                raise ValueError(f"指定された期間 {specific_period} がデータに存在しません。利用可能な期間: {sorted(available_periods)}")
            target_period = specific_period
            period_info = f" (指定期間: {specific_period})"
        else:
            # 期間未指定の場合は最新期間を使用
            target_period = df.index.get_level_values(1).max()
            period_info = f" (最新期間: {target_period})"
        
        # 対象期間のデータを抽出
        df = df.xs(target_period, level=1)
        
        category_companies = {}
        
        print(f"=== {table_name} カテゴリー分析{period_info} ===")
        print(f"総企業数: {len(df.index.unique())}")
        print(f"総カテゴリー数: {len(df.columns)}")
        print(f"閾値: {threshold}\n")
        
        for category in df.columns:
            # 閾値以上の値を持つ企業のみ抽出
            companies_in_category = df[df[category] >= threshold].index
            
            if len(companies_in_category) > 0:
                # 特定期間のデータなので、企業IDのみ
                entity_ids = companies_in_category.tolist()
                fterm_2s = [target_period] * len(entity_ids)
                
                # DataFrameに変換
                category_df = pd.DataFrame({
                    'FACTSET_ENTITY_ID': entity_ids,
                    'FTERM_2': fterm_2s,
                    'CATEGORY_VALUE': df.loc[companies_in_category, category].values
                })
                
                # 売上高情報の追加（財務データがある場合）
                if self.financial_data is not None:
                    category_df = self._add_financial_info(category_df, target_period)
                
                # カテゴリー値でソート（降順）
                category_df = category_df.sort_values('CATEGORY_VALUE', ascending=False)
                
                category_companies[category] = category_df
        
        return category_companies
    
    def _add_financial_info(self, category_df: pd.DataFrame, target_period: int) -> pd.DataFrame:
        """財務情報の追加"""
        if self.financial_data is None:
            return category_df
        
        # 対象期間の財務データに制限
        financial_data_filtered = self.financial_data[
            self.financial_data['FTERM_2'] == target_period
        ].drop_duplicates(subset=['FACTSET_ENTITY_ID'])
        merge_cols = ['FACTSET_ENTITY_ID']
        
        # 財務データとマージ
        merged = category_df.merge(
            financial_data_filtered[['FACTSET_ENTITY_ID', 'FTERM_2', 'FF_CO_NAME', 
                                   'FF_SALES', 'FF_OPER_INC', 'FF_ASSETS', 'CURRENCY']],
            on=merge_cols,
            how='left'
        )
        
        # 売上高でソート（降順）
        merged = merged.sort_values(['CATEGORY_VALUE', 'FF_SALES'], 
                                  ascending=[False, False])
        
        return merged
    
    def get_category_summary(self, category_companies: Dict[str, pd.DataFrame], 
                           top_n: int = 10) -> pd.DataFrame:
        """カテゴリー別サマリーの取得"""
        summary_data = []
        
        for category, companies_df in category_companies.items():
            total_companies = len(companies_df['FACTSET_ENTITY_ID'].unique())
            
            if self.financial_data is not None and 'FF_SALES' in companies_df.columns:
                # 売上高がある場合の統計
                total_sales = companies_df['FF_SALES'].sum()
                avg_sales = companies_df['FF_SALES'].mean()
                median_sales = companies_df['FF_SALES'].median()
                max_dependency = companies_df['CATEGORY_VALUE'].max()
                avg_dependency = companies_df['CATEGORY_VALUE'].mean()
                
                summary_data.append({
                    'Category': category,
                    'Company_Count': total_companies,
                    'Total_Sales': total_sales,
                    'Avg_Sales': avg_sales,
                    'Median_Sales': median_sales,
                    'Max_Dependency': max_dependency,
                    'Avg_Dependency': avg_dependency
                })
            else:
                # 売上高がない場合の統計
                max_dependency = companies_df['CATEGORY_VALUE'].max()
                avg_dependency = companies_df['CATEGORY_VALUE'].mean()
                
                summary_data.append({
                    'Category': category,
                    'Company_Count': total_companies,
                    'Max_Dependency': max_dependency,
                    'Avg_Dependency': avg_dependency
                })
        
        summary_df = pd.DataFrame(summary_data)
        return summary_df.sort_values('Company_Count', ascending=False)
    
    def plot_category_distribution(self, summary_df: pd.DataFrame, top_n: int = 20) -> go.Figure:
        """カテゴリー別企業数分布の可視化"""
        top_categories = summary_df.head(top_n)
        
        fig = go.Figure(data=[
            go.Bar(
                x=top_categories['Category'],
                y=top_categories['Company_Count'],
                text=top_categories['Company_Count'],
                textposition='auto',
                marker_color='lightblue'
            )
        ])
        
        fig.update_layout(
            title=f'Top {top_n} カテゴリー別企業数分布',
            xaxis_title='カテゴリー',
            yaxis_title='企業数',
            xaxis_tickangle=-45,
            height=600,
            font=dict(size=12)
        )
        
        return fig
    
    def plot_sales_vs_dependency(self, category_companies: Dict[str, pd.DataFrame], 
                                category: str, top_n: int = 20) -> go.Figure:
        """特定カテゴリーの売上高vs依存度の散布図"""
        if category not in category_companies:
            raise ValueError(f"Category {category} not found")
        
        df = category_companies[category].head(top_n)
        
        if 'FF_SALES' not in df.columns:
            print("売上高データがありません")
            return None
        
        fig = go.Figure()
        
        fig.add_trace(go.Scatter(
            x=df['CATEGORY_VALUE'],
            y=df['FF_SALES'],
            mode='markers+text',
            text=df['FF_CO_NAME'].str[:10] + '...',  # 会社名を短縮
            textposition='top center',
            marker=dict(
                size=10,
                color=df['CATEGORY_VALUE'],
                colorscale='Viridis',
                showscale=True,
                colorbar=dict(title="依存度")
            ),
            hovertemplate=
            '<b>%{text}</b><br>' +
            '依存度: %{x:.3f}<br>' +
            '売上高: %{y:,.0f}<br>' +
            '<extra></extra>'
        ))
        
        fig.update_layout(
            title=f'{category} カテゴリー: 売上高 vs 依存度 (Top {top_n})',
            xaxis_title='カテゴリー依存度',
            yaxis_title='売上高',
            height=600,
            font=dict(size=12)
        )
        
        return fig
    
    def plot_dependency_heatmap(self, table_name: str = 'consol_l5', 
                               top_companies: int = 20, top_categories: int = 15,
                               specific_period: Optional[int] = None) -> go.Figure:
        """依存度ヒートマップの作成"""
        if table_name not in self.pivot_tables:
            raise ValueError(f"Table {table_name} not found")
        
        df = self.pivot_tables[table_name]
        
        # 期間の選択
        if specific_period is not None:
            available_periods = df.index.get_level_values(1).unique()
            if specific_period not in available_periods:
                raise ValueError(f"指定された期間 {specific_period} がデータに存在しません。利用可能な期間: {sorted(available_periods)}")
            target_period = specific_period
            period_df = df.xs(target_period, level=1)
        else:
            # 最新期間のデータを使用
            target_period = df.index.get_level_values(1).max()
            period_df = df.xs(target_period, level=1)
        
        # 売上高情報を追加してトップ企業を選択
        if self.financial_data is not None:
            # 対象期間の財務データ
            period_financial = self.financial_data[
                self.financial_data['FTERM_2'] == target_period
            ].drop_duplicates(subset=['FACTSET_ENTITY_ID'])
            
            # マージして売上高順にソート
            merged = period_df.reset_index().merge(
                period_financial[['FACTSET_ENTITY_ID', 'FF_CO_NAME', 'FF_SALES']],
                on='FACTSET_ENTITY_ID',
                how='left'
            )
            
            top_entities = merged.nlargest(top_companies, 'FF_SALES')['FACTSET_ENTITY_ID'].tolist()
        else:
            # 売上高データがない場合は、依存度の合計が高い企業を選択
            top_entities = period_df.sum(axis=1).nlargest(top_companies).index.tolist()
        
        # トップカテゴリーを選択（平均依存度が高い順）
        top_categories_list = period_df.mean().nlargest(top_categories).index.tolist()
        
        # ヒートマップ用データの作成
        heatmap_data = period_df.loc[top_entities, top_categories_list]
        
        # 企業名を取得（可能な場合）
        if self.financial_data is not None:
            entity_names = []
            for entity_id in top_entities:
                name_data = self.financial_data[
                    self.financial_data['FACTSET_ENTITY_ID'] == entity_id
                ]['FF_CO_NAME'].iloc[0] if len(self.financial_data[
                    self.financial_data['FACTSET_ENTITY_ID'] == entity_id
                ]) > 0 else entity_id
                entity_names.append(name_data[:20] + '...' if len(str(name_data)) > 20 else str(name_data))
        else:
            entity_names = [str(eid)[:20] + '...' if len(str(eid)) > 20 else str(eid) for eid in top_entities]
        
        fig = go.Figure(data=go.Heatmap(
            z=heatmap_data.values,
            x=[col[:30] + '...' if len(col) > 30 else col for col in top_categories_list],
            y=entity_names,
            colorscale='Viridis',
            hoverongaps=False,
            hovertemplate=
            '企業: %{y}<br>' +
            'カテゴリー: %{x}<br>' +
            '依存度: %{z:.3f}<br>' +
            '<extra></extra>'
        ))
        
        fig.update_layout(
            title=f'{table_name} 依存度ヒートマップ (期間: {target_period})',
            xaxis_title='カテゴリー',
            yaxis_title='企業',
            height=800,
            font=dict(size=10)
        )
        
        return fig


In [ ]:
# 分析実行のメイン関数（期間指定対応版）
def run_category_analysis(results_dict: Dict, specific_period: Optional[int] = None) -> Dict:
    """
    カテゴリー分析の実行
    
    Args:
        results_dict: main()関数の実行結果
        specific_period: 特定期間の指定（例：202403）。Noneの場合は最新期間を自動使用
        
    Returns:
        分析結果辞書
    """
    period_text = f"指定期間: {specific_period}" if specific_period else "最新期間（自動選択）"
    print(f"=== カテゴリー分析開始（{period_text}） ===\n")
    
    # アナライザーの初期化
    analyzer = CategoryAnalyzer(
        pivot_tables_dict=results_dict['processed_pivot_tables'],
        financial_data=results_dict['data']['financial']
    )
    
    # 1. consol_l5のカテゴリー別企業分析
    print("1. consol_l5 カテゴリー別企業分析")
    category_companies = analyzer.analyze_category_companies(
        table_name='consol_l5', 
        threshold=0.01,  # 1%以上の依存度を持つ企業
        specific_period=specific_period  # 特定期間指定
    )
    
    # 2. カテゴリーサマリーの取得
    print("\n2. カテゴリーサマリーの作成")
    summary_df = analyzer.get_category_summary(category_companies)
    print(f"分析対象カテゴリー数: {len(summary_df)}")
    
    # 3. 上位カテゴリーの表示
    print("\n=== 企業数上位10カテゴリー ===")
    print(summary_df.head(10)[['Category', 'Company_Count', 'Total_Sales', 'Avg_Dependency']].to_string(index=False))
    
    # 4. 可視化の作成と表示
    print("\n3. 可視化の作成")
    
    # カテゴリー別企業数分布
    fig1 = analyzer.plot_category_distribution(summary_df, top_n=20)
    fig1.show()
    
    # 依存度ヒートマップ
    fig2 = analyzer.plot_dependency_heatmap(table_name='consol_l5', top_companies=15, top_categories=12, specific_period=specific_period)
    fig2.show()
    
    # 5. 特定カテゴリーの詳細分析（上位カテゴリーから選択）
    if len(summary_df) > 0:
        top_category = summary_df.iloc[0]['Category']
        print(f"\n4. 特定カテゴリー詳細分析: {top_category}")
        
        # 売上高vs依存度の散布図
        fig3 = analyzer.plot_sales_vs_dependency(category_companies, top_category, top_n=15)
        if fig3 is not None:
            fig3.show()
        
        # そのカテゴリーの上位企業表示
        if top_category in category_companies:
            top_companies_in_category = category_companies[top_category].head(10)
            print(f"\n{top_category} カテゴリー上位企業:")
            display_cols = ['FF_CO_NAME', 'CATEGORY_VALUE', 'FF_SALES', 'CURRENCY']
            available_cols = [col for col in display_cols if col in top_companies_in_category.columns]
            print(top_companies_in_category[available_cols].to_string(index=False))
    
    print("\n=== カテゴリー分析完了 ===")
    
    return {
        'analyzer': analyzer,
        'category_companies': category_companies,
        'summary_df': summary_df,
        'figures': [fig1, fig2, fig3] if 'fig3' in locals() else [fig1, fig2]
    }

# 使用例とデモ
def demo_analysis(specific_period: Optional[int] = None):
    """
    データ分析のデモ実行
    
    Args:
        specific_period: 特定期間の指定（例：202403）。Noneの場合は最新期間を自動使用
    """
    period_text = f"指定期間: {specific_period}" if specific_period else "最新期間（自動選択）"
    print(f"=== FactSet データ分析デモ（{period_text}） ===\n")
    
    # 1. メインの処理を実行
    print("1. メインデータ処理の実行...")
    results = main()
    
    # 2. カテゴリー分析の実行
    print("\n2. カテゴリー分析の実行...")
    analysis_results = run_category_analysis(results, specific_period=specific_period)
    
    # 3. 特定カテゴリーの詳細分析例
    print("\n3. 詳細分析の例...")
    if len(analysis_results['summary_df']) > 0:
        # 上位3カテゴリーを詳細分析
        top_3_categories = analysis_results['summary_df'].head(3)['Category'].tolist()
        
        for i, category in enumerate(top_3_categories):
            print(f"\n--- {i+1}. {category} の詳細 ---")
            explore_specific_category(
                analysis_results['category_companies'], 
                category, 
                top_n=5
            )
        
        # 4. カテゴリー比較
        print("\n4. カテゴリー比較...")
        comparison_fig = compare_categories(
            analysis_results['summary_df'],
            analysis_results['category_companies'],
            top_3_categories
        )
        if comparison_fig:
            comparison_fig.show()
        
        # 5. クロスカテゴリー企業分析
        print("\n5. クロスカテゴリー企業分析...")
        cross_df = get_cross_category_companies(
            analysis_results['category_companies'],
            top_3_categories[:2],  # 上位2カテゴリーで分析
            min_dependency=0.03
        )
        
        if len(cross_df) > 0:
            print("\n複数カテゴリーに属する企業（上位5社）:")
            print(cross_df.head(10).to_string(index=False))
    
    print("\n=== デモ完了 ===")
    return results, analysis_results

# 個別実行用の簡易版
def quick_analysis():
    """
    簡易分析（既にresultsがある場合）
    """
    print("簡易分析を実行するには、まずmain()を実行してresultsを取得してください:")
    print("results = main()")
    print("analysis_results = run_category_analysis(results)")
    print("\n特定カテゴリーの分析:")
    print("explore_specific_category(analysis_results['category_companies'], 'カテゴリー名')")

# 実行コメント
print("分析実行方法（期間指定対応版）:")
print("1. 最新期間での自動デモ: demo_analysis()")
print("2. 特定期間でのデモ: demo_analysis(specific_period=202403)")
print("3. 個別実行: results = main() -> analysis_results = run_category_analysis(results, specific_period=202403)")
print("4. 詳細分析: explore_specific_category(category_companies, 'カテゴリー名')")
print("\n注意: specific_period が指定されない場合は自動的に FTERM_2 の最新期間が使用されます。")


In [ ]:
# 可視化用ライブラリの追加インポート
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.offline as pyo

# インタラクティブな表示のため
pyo.init_notebook_mode(connected=True)


In [ ]:
# 分析実行のメイン関数
def run_category_analysis(results_dict: Dict) -> Dict:
    """
    カテゴリー分析の実行
    
    Args:
        results_dict: main()関数の実行結果
        
    Returns:
        分析結果辞書
    """
    print("=== カテゴリー分析開始 ===\n")
    
    # アナライザーの初期化
    analyzer = CategoryAnalyzer(
        pivot_tables_dict=results_dict['processed_pivot_tables'],
        financial_data=results_dict['data']['financial']
    )
    
    # 1. consol_l5のカテゴリー別企業分析（最新期間のみ）
    print("1. consol_l5 カテゴリー別企業分析")
    category_companies = analyzer.analyze_category_companies(
        table_name='consol_l5', 
        threshold=0.01,  # 1%以上の依存度を持つ企業
        use_latest_period_only=True  # 最新期間のみ使用
    )
    
    # 2. カテゴリーサマリーの取得
    print("\n2. カテゴリーサマリーの作成")
    summary_df = analyzer.get_category_summary(category_companies)
    print(f"分析対象カテゴリー数: {len(summary_df)}")
    
    # 3. 上位カテゴリーの表示
    print("\n=== 企業数上位10カテゴリー ===")
    print(summary_df.head(10)[['Category', 'Company_Count', 'Total_Sales', 'Avg_Dependency']].to_string(index=False))
    
    # 4. 可視化の作成と表示
    print("\n3. 可視化の作成")
    
    # カテゴリー別企業数分布
    fig1 = analyzer.plot_category_distribution(summary_df, top_n=20)
    fig1.show()
    
    # 依存度ヒートマップ
    fig2 = analyzer.plot_dependency_heatmap(table_name='consol_l5', top_companies=15, top_categories=12)
    fig2.show()
    
    # 5. 特定カテゴリーの詳細分析（上位カテゴリーから選択）
    if len(summary_df) > 0:
        top_category = summary_df.iloc[0]['Category']
        print(f"\n4. 特定カテゴリー詳細分析: {top_category}")
        
        # 売上高vs依存度の散布図
        fig3 = analyzer.plot_sales_vs_dependency(category_companies, top_category, top_n=15)
        if fig3 is not None:
            fig3.show()
        
        # そのカテゴリーの上位企業表示
        if top_category in category_companies:
            top_companies_in_category = category_companies[top_category].head(10)
            print(f"\n{top_category} カテゴリー上位企業:")
            display_cols = ['FF_CO_NAME', 'CATEGORY_VALUE', 'FF_SALES', 'CURRENCY']
            available_cols = [col for col in display_cols if col in top_companies_in_category.columns]
            print(top_companies_in_category[available_cols].to_string(index=False))
    
    print("\n=== カテゴリー分析完了 ===")
    
    return {
        'analyzer': analyzer,
        'category_companies': category_companies,
        'summary_df': summary_df,
        'figures': [fig1, fig2, fig3] if 'fig3' in locals() else [fig1, fig2]
    }


In [ ]:
# 詳細分析用のユーティリティ関数
def explore_specific_category(category_companies: Dict[str, pd.DataFrame], 
                            category_name: str, 
                            top_n: int = 20) -> pd.DataFrame:
    """
    特定カテゴリーの詳細探索
    
    Args:
        category_companies: カテゴリー別企業辞書
        category_name: 分析対象カテゴリー名
        top_n: 表示する企業数
        
    Returns:
        詳細データフレーム
    """
    if category_name not in category_companies:
        print(f"カテゴリー '{category_name}' が見つかりません。")
        print("利用可能なカテゴリー:")
        for i, cat in enumerate(sorted(category_companies.keys())[:10]):
            print(f"  {i+1}. {cat}")
        return pd.DataFrame()
    
    df = category_companies[category_name].head(top_n)
    
    print(f"=== {category_name} カテゴリー詳細 ===")
    print(f"該当企業数: {len(category_companies[category_name])}")
    print(f"表示企業数: {min(top_n, len(df))}")
    
    if 'FF_SALES' in df.columns:
        print(f"総売上高: {df['FF_SALES'].sum():,.0f}")
        print(f"平均売上高: {df['FF_SALES'].mean():,.0f}")
        print(f"平均依存度: {df['CATEGORY_VALUE'].mean():.3f}")
    
    print(f"\n上位企業:")
    display_cols = ['FF_CO_NAME', 'CATEGORY_VALUE', 'FF_SALES', 'CURRENCY', 'FTERM_2']
    available_cols = [col for col in display_cols if col in df.columns]
    print(df[available_cols].to_string(index=False))
    
    return df

def compare_categories(summary_df: pd.DataFrame, 
                      category_companies: Dict[str, pd.DataFrame],
                      categories: List[str]) -> go.Figure:
    """
    複数カテゴリーの比較可視化
    
    Args:
        summary_df: カテゴリーサマリー
        category_companies: カテゴリー別企業辞書
        categories: 比較対象カテゴリーリスト
        
    Returns:
        比較グラフ
    """
    fig = make_subplots(
        rows=2, cols=2,
        subplot_titles=('企業数', '総売上高', '平均依存度', '最大依存度'),
        specs=[[{'type': 'bar'}, {'type': 'bar'}],
               [{'type': 'bar'}, {'type': 'bar'}]]
    )
    
    # フィルタリング
    filtered_summary = summary_df[summary_df['Category'].isin(categories)]
    
    if len(filtered_summary) == 0:
        print("指定されたカテゴリーが見つかりません")
        return None
    
    # グラフの作成
    fig.add_trace(
        go.Bar(x=filtered_summary['Category'], y=filtered_summary['Company_Count'], name='企業数'),
        row=1, col=1
    )
    
    if 'Total_Sales' in filtered_summary.columns:
        fig.add_trace(
            go.Bar(x=filtered_summary['Category'], y=filtered_summary['Total_Sales'], name='総売上高'),
            row=1, col=2
        )
    
    fig.add_trace(
        go.Bar(x=filtered_summary['Category'], y=filtered_summary['Avg_Dependency'], name='平均依存度'),
        row=2, col=1
    )
    
    fig.add_trace(
        go.Bar(x=filtered_summary['Category'], y=filtered_summary['Max_Dependency'], name='最大依存度'),
        row=2, col=2
    )
    
    fig.update_layout(
        title='カテゴリー比較',
        height=800,
        showlegend=False
    )
    
    return fig

def get_cross_category_companies(category_companies: Dict[str, pd.DataFrame],
                               categories: List[str],
                               min_dependency: float = 0.05) -> pd.DataFrame:
    """
    複数カテゴリーにまたがる企業の分析
    
    Args:
        category_companies: カテゴリー別企業辞書
        categories: 対象カテゴリーリスト
        min_dependency: 最小依存度閾値
        
    Returns:
        クロスカテゴリー企業データフレーム
    """
    cross_companies = []
    
    for category in categories:
        if category in category_companies:
            df = category_companies[category]
            df = df[df['CATEGORY_VALUE'] >= min_dependency]
            df['CATEGORY'] = category
            cross_companies.append(df[['FACTSET_ENTITY_ID', 'FF_CO_NAME', 
                                     'CATEGORY_VALUE', 'CATEGORY', 'FF_SALES']])
    
    if not cross_companies:
        return pd.DataFrame()
    
    combined = pd.concat(cross_companies, ignore_index=True)
    
    # 複数カテゴリーに属する企業の抽出
    entity_counts = combined['FACTSET_ENTITY_ID'].value_counts()
    multi_category_entities = entity_counts[entity_counts > 1].index
    
    result = combined[combined['FACTSET_ENTITY_ID'].isin(multi_category_entities)]
    result = result.sort_values(['FACTSET_ENTITY_ID', 'CATEGORY_VALUE'], ascending=[True, False])
    
    print(f"複数カテゴリーに属する企業数: {len(multi_category_entities)}")
    print(f"対象カテゴリー: {', '.join(categories)}")
    
    return result

print("詳細分析用関数が定義されました。")


In [ ]:
# 使用例とデモ
def demo_analysis():
    """
    データ分析のデモ実行
    """
    print("=== FactSet データ分析デモ ===\n")
    
    # 1. メインの処理を実行
    print("1. メインデータ処理の実行...")
    results = main()
    
    # 2. カテゴリー分析の実行
    print("\n2. カテゴリー分析の実行...")
    analysis_results = run_category_analysis(results)
    
    # 3. 特定カテゴリーの詳細分析例
    print("\n3. 詳細分析の例...")
    if len(analysis_results['summary_df']) > 0:
        # 上位3カテゴリーを詳細分析
        top_3_categories = analysis_results['summary_df'].head(3)['Category'].tolist()
        
        for i, category in enumerate(top_3_categories):
            print(f"\n--- {i+1}. {category} の詳細 ---")
            explore_specific_category(
                analysis_results['category_companies'], 
                category, 
                top_n=5
            )
        
        # 4. カテゴリー比較
        print("\n4. カテゴリー比較...")
        comparison_fig = compare_categories(
            analysis_results['summary_df'],
            analysis_results['category_companies'],
            top_3_categories
        )
        if comparison_fig:
            comparison_fig.show()
        
        # 5. クロスカテゴリー企業分析
        print("\n5. クロスカテゴリー企業分析...")
        cross_df = get_cross_category_companies(
            analysis_results['category_companies'],
            top_3_categories[:2],  # 上位2カテゴリーで分析
            min_dependency=0.03
        )
        
        if len(cross_df) > 0:
            print("\n複数カテゴリーに属する企業（上位5社）:")
            print(cross_df.head(10).to_string(index=False))
    
    print("\n=== デモ完了 ===")
    return results, analysis_results

# 個別実行用の簡易版
def quick_analysis():
    """
    簡易分析（既にresultsがある場合）
    """
    print("簡易分析を実行するには、まずmain()を実行してresultsを取得してください:")
    print("results = main()")
    print("analysis_results = run_category_analysis(results)")
    print("\n特定カテゴリーの分析:")
    print("explore_specific_category(analysis_results['category_companies'], 'カテゴリー名')")

# 実行コメント
print("分析実行方法:")
print("1. 完全デモ: demo_analysis()")
print("2. 個別実行: results = main() -> analysis_results = run_category_analysis(results)")
print("3. 詳細分析: explore_specific_category(category_companies, 'カテゴリー名')")


In [ ]:
# カテゴリー間相関分析とマルチコリニアリティ検出クラス
import numpy as np
import pandas as pd
from scipy.cluster.hierarchy import linkage, dendrogram
from scipy.spatial.distance import pdist
from sklearn.preprocessing import StandardScaler
from statsmodels.stats.outliers_influence import variance_inflation_factor
import networkx as nx
import plotly.figure_factory as ff
from typing import Tuple, List

class MulticollinearityAnalyzer:
    """カテゴリー間相関分析とマルチコリニアリティ検出クラス"""
    
    def __init__(self, pivot_tables_dict: Dict[str, pd.DataFrame]):
        self.pivot_tables = pivot_tables_dict
        self.correlation_matrix = None
        self.high_corr_pairs = None
        
    def calculate_correlation_matrix(self, table_name: str = 'consol_l5', 
                                   specific_period: Optional[int] = None,
                                   method: str = 'pearson') -> pd.DataFrame:
        """
        カテゴリー間の相関行列を計算
        
        Args:
            table_name: 分析対象のテーブル名
            specific_period: 特定期間の指定。Noneの場合は最新期間を使用
            method: 相関係数の計算方法（'pearson', 'spearman', 'kendall'）
            
        Returns:
            相関行列
        """
        if table_name not in self.pivot_tables:
            raise ValueError(f"Table {table_name} not found in pivot_tables")
        
        df = self.pivot_tables[table_name]
        
        # 期間の選択
        if specific_period is not None:
            available_periods = df.index.get_level_values(1).unique()
            if specific_period not in available_periods:
                raise ValueError(f"指定された期間 {specific_period} がデータに存在しません。")
            target_period = specific_period
            period_df = df.xs(target_period, level=1)
        else:
            target_period = df.index.get_level_values(1).max()
            period_df = df.xs(target_period, level=1)
        
        # 相関行列の計算
        self.correlation_matrix = period_df.corr(method=method)
        
        print(f"相関行列計算完了 (期間: {target_period}, 方法: {method})")
        print(f"カテゴリー数: {len(self.correlation_matrix.columns)}")
        
        return self.correlation_matrix
    
    def find_high_correlation_pairs(self, threshold: float = 0.7, 
                                  exclude_self: bool = True) -> pd.DataFrame:
        """
        高相関ペアを抽出
        
        Args:
            threshold: 相関係数の閾値
            exclude_self: 自分自身との相関（1.0）を除外するか
            
        Returns:
            高相関ペアのデータフレーム
        """
        if self.correlation_matrix is None:
            raise ValueError("先に calculate_correlation_matrix() を実行してください")
        
        # 上三角行列のみを取得（重複除去）
        corr_matrix = self.correlation_matrix.copy()
        if exclude_self:
            np.fill_diagonal(corr_matrix.values, np.nan)
        
        # 上三角のマスクを作成
        mask = np.triu(np.ones_like(corr_matrix), k=1).astype(bool)
        
        # 高相関ペアを抽出
        high_corr_indices = np.where((np.abs(corr_matrix.values) >= threshold) & mask)
        
        pairs_data = []
        for i, j in zip(high_corr_indices[0], high_corr_indices[1]):
            category1 = corr_matrix.index[i]
            category2 = corr_matrix.columns[j]
            correlation = corr_matrix.iloc[i, j]
            
            pairs_data.append({
                'Category1': category1,
                'Category2': category2,
                'Correlation': correlation,
                'Abs_Correlation': abs(correlation)
            })
        
        self.high_corr_pairs = pd.DataFrame(pairs_data).sort_values('Abs_Correlation', ascending=False)
        
        print(f"高相関ペア（|r| >= {threshold}）: {len(self.high_corr_pairs)} 組")
        
        return self.high_corr_pairs
    
    def calculate_vif(self, table_name: str = 'consol_l5', 
                     specific_period: Optional[int] = None,
                     vif_threshold: float = 5.0) -> pd.DataFrame:
        """
        VIF（分散拡大係数）を計算してマルチコリニアリティを評価
        
        Args:
            table_name: 分析対象のテーブル名
            specific_period: 特定期間の指定
            vif_threshold: 問題ありとするVIFの閾値
            
        Returns:
            VIF結果のデータフレーム
        """
        if table_name not in self.pivot_tables:
            raise ValueError(f"Table {table_name} not found")
        
        df = self.pivot_tables[table_name]
        
        # 期間の選択
        if specific_period is not None:
            target_period = specific_period
            period_df = df.xs(target_period, level=1)
        else:
            target_period = df.index.get_level_values(1).max()
            period_df = df.xs(target_period, level=1)
        
        # NaNや無限大値を除去
        clean_df = period_df.fillna(0)
        clean_df = clean_df.replace([np.inf, -np.inf], 0)
        
        # 分散が0のカラムを除去
        non_zero_var_cols = clean_df.var() > 1e-10
        clean_df = clean_df.loc[:, non_zero_var_cols]
        
        print(f"VIF計算対象カテゴリー数: {len(clean_df.columns)}")
        
        # VIF計算
        vif_data = []
        for i, col in enumerate(clean_df.columns):
            try:
                vif_value = variance_inflation_factor(clean_df.values, i)
                vif_data.append({
                    'Category': col,
                    'VIF': vif_value,
                    'High_VIF': vif_value > vif_threshold
                })
            except Exception as e:
                print(f"VIF計算エラー ({col}): {e}")
                vif_data.append({
                    'Category': col,
                    'VIF': np.nan,
                    'High_VIF': False
                })
        
        vif_df = pd.DataFrame(vif_data).sort_values('VIF', ascending=False)
        
        high_vif_count = vif_df['High_VIF'].sum()
        print(f"高VIF（> {vif_threshold}）カテゴリー数: {high_vif_count}")
        
        return vif_df

print("MulticollinearityAnalyzer クラス（パート1）が定義されました。")


In [ ]:
# MulticollinearityAnalyzer クラスの可視化メソッド（続き）

# クラスに新しいメソッドを追加
def plot_correlation_heatmap(self, figsize: Tuple[int, int] = (12, 10),
                           show_values: bool = False,
                           high_corr_threshold: float = 0.7) -> go.Figure:
    """
    相関行列のヒートマップを作成
    
    Args:
        figsize: 図のサイズ
        show_values: 相関係数の値を表示するか
        high_corr_threshold: 高相関として強調する閾値
        
    Returns:
        plotlyの図オブジェクト
    """
    if self.correlation_matrix is None:
        raise ValueError("先に calculate_correlation_matrix() を実行してください")
    
    # カテゴリー名を短縮
    short_names = [name[:20] + '...' if len(name) > 20 else name 
                  for name in self.correlation_matrix.columns]
    
    # ヒートマップの作成
    fig = go.Figure(data=go.Heatmap(
        z=self.correlation_matrix.values,
        x=short_names,
        y=short_names,
        colorscale='RdBu_r',
        zmid=0,
        zmin=-1,
        zmax=1,
        text=np.round(self.correlation_matrix.values, 3) if show_values else None,
        texttemplate='%{text}' if show_values else None,
        textfont={'size': 8},
        hovertemplate=
        'カテゴリー1: %{y}<br>' +
        'カテゴリー2: %{x}<br>' +
        '相関係数: %{z:.3f}<br>' +
        '<extra></extra>'
    ))
    
    fig.update_layout(
        title='カテゴリー間相関行列',
        xaxis_title='カテゴリー',
        yaxis_title='カテゴリー',
        width=figsize[0] * 80,
        height=figsize[1] * 80,
        font=dict(size=10)
    )
    
    return fig

def plot_correlation_network(self, threshold: float = 0.7,
                           layout: str = 'spring') -> go.Figure:
    """
    高相関カテゴリーのネットワーク図を作成
    
    Args:
        threshold: ネットワークに含める相関の閾値
        layout: ネットワークレイアウト
        
    Returns:
        plotlyの図オブジェクト
    """
    if self.correlation_matrix is None:
        raise ValueError("先に calculate_correlation_matrix() を実行してください")
    
    # ネットワークグラフの作成
    G = nx.Graph()
    
    # ノードの追加
    categories = self.correlation_matrix.columns.tolist()
    G.add_nodes_from(categories)
    
    # エッジの追加（高相関のペアのみ）
    for i in range(len(categories)):
        for j in range(i + 1, len(categories)):
            corr = abs(self.correlation_matrix.iloc[i, j])
            if corr >= threshold:
                G.add_edge(categories[i], categories[j], weight=corr)
    
    if len(G.edges()) == 0:
        print(f"閾値 {threshold} 以上の相関を持つペアが見つかりません")
        return None
    
    # レイアウトの計算
    if layout == 'spring':
        pos = nx.spring_layout(G, k=3, iterations=50)
    elif layout == 'circular':
        pos = nx.circular_layout(G)
    else:
        pos = nx.random_layout(G)
    
    # エッジの座標
    edge_x = []
    edge_y = []
    edge_weights = []
    
    for edge in G.edges():
        x0, y0 = pos[edge[0]]
        x1, y1 = pos[edge[1]]
        edge_x.extend([x0, x1, None])
        edge_y.extend([y0, y1, None])
        edge_weights.append(G[edge[0]][edge[1]]['weight'])
    
    # ノードの座標
    node_x = [pos[node][0] for node in G.nodes()]
    node_y = [pos[node][1] for node in G.nodes()]
    node_text = [name[:15] + '...' if len(name) > 15 else name for name in G.nodes()]
    
    # 図の作成
    fig = go.Figure()
    
    # エッジの追加
    fig.add_trace(go.Scatter(
        x=edge_x, y=edge_y,
        line=dict(width=2, color='lightgray'),
        hoverinfo='none',
        mode='lines',
        name='相関'
    ))
    
    # ノードの追加
    fig.add_trace(go.Scatter(
        x=node_x, y=node_y,
        mode='markers+text',
        text=node_text,
        textposition='middle center',
        marker=dict(
            size=30,
            color='lightblue',
            line=dict(width=2, color='darkblue')
        ),
        hovertemplate='カテゴリー: %{text}<extra></extra>',
        name='カテゴリー'
    ))
    
    fig.update_layout(
        title=f'高相関カテゴリーネットワーク (|r| >= {threshold})',
        showlegend=False,
        hovermode='closest',
        margin=dict(b=20, l=5, r=5, t=40),
        annotations=[
            dict(
                text=f"ノード数: {len(G.nodes())}, エッジ数: {len(G.edges())}",
                showarrow=False,
                xref='paper', yref='paper',
                x=0.005, y=-0.002,
                xanchor='left', yanchor='bottom',
                font=dict(size=12)
            )
        ],
        xaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
        yaxis=dict(showgrid=False, zeroline=False, showticklabels=False)
    )
    
    return fig

def plot_dendrogram(self, method: str = 'ward', 
                   metric: str = 'euclidean') -> go.Figure:
    """
    カテゴリーの階層クラスタリング結果をデンドログラムで表示
    
    Args:
        method: クラスタリング手法
        metric: 距離計算方法
        
    Returns:
        plotlyの図オブジェクト
    """
    if self.correlation_matrix is None:
        raise ValueError("先に calculate_correlation_matrix() を実行してください")
    
    # 相関行列から距離行列を作成（1 - |correlation|）
    distance_matrix = 1 - np.abs(self.correlation_matrix.values)
    
    # 階層クラスタリング
    linkage_matrix = linkage(pdist(distance_matrix, metric=metric), method=method)
    
    # デンドログラムの作成
    labels = [name[:20] + '...' if len(name) > 20 else name 
             for name in self.correlation_matrix.columns]
    
    fig = ff.create_dendrogram(
        distance_matrix,
        labels=labels,
        linkagefun=lambda x: linkage(x, method=method, metric=metric)
    )
    
    fig.update_layout(
        title='カテゴリー階層クラスタリング（相関ベース）',
        xaxis_title='カテゴリー',
        yaxis_title='距離 (1 - |相関係数|)',
        height=600
    )
    
    return fig

# メソッドをクラスに追加
MulticollinearityAnalyzer.plot_correlation_heatmap = plot_correlation_heatmap
MulticollinearityAnalyzer.plot_correlation_network = plot_correlation_network
MulticollinearityAnalyzer.plot_dendrogram = plot_dendrogram

print("MulticollinearityAnalyzer クラスの可視化メソッドが追加されました。")


In [ ]:
# MulticollinearityAnalyzer クラスのレポート生成メソッドと実行関数

def generate_multicollinearity_report(self, table_name: str = 'consol_l5',
                                    specific_period: Optional[int] = None,
                                    corr_threshold: float = 0.7,
                                    vif_threshold: float = 5.0) -> Dict:
    """
    マルチコリニアリティの総合レポートを生成
    
    Args:
        table_name: 分析対象のテーブル名
        specific_period: 特定期間の指定
        corr_threshold: 高相関の閾値
        vif_threshold: 高VIFの閾値
        
    Returns:
        分析結果の辞書
    """
    print("=== マルチコリニアリティ分析レポート ===\n")
    
    # 1. 相関行列の計算
    print("1. 相関行列の計算...")
    corr_matrix = self.calculate_correlation_matrix(table_name, specific_period)
    
    # 2. 高相関ペアの抽出
    print("\n2. 高相関ペアの抽出...")
    high_corr_pairs = self.find_high_correlation_pairs(corr_threshold)
    
    # 3. VIF計算
    print("\n3. VIF計算...")
    vif_results = self.calculate_vif(table_name, specific_period, vif_threshold)
    
    # 4. 結果のサマリー
    print(f"\n=== サマリー ===")
    print(f"総カテゴリー数: {len(corr_matrix.columns)}")
    print(f"高相関ペア数 (|r| >= {corr_threshold}): {len(high_corr_pairs)}")
    print(f"高VIFカテゴリー数 (VIF > {vif_threshold}): {vif_results['High_VIF'].sum()}")
    
    if len(high_corr_pairs) > 0:
        print(f"\n最高相関ペア:")
        top_pair = high_corr_pairs.iloc[0]
        print(f"  {top_pair['Category1']} - {top_pair['Category2']}: {top_pair['Correlation']:.3f}")
    
    if vif_results['High_VIF'].sum() > 0:
        print(f"\n最高VIFカテゴリー:")
        top_vif = vif_results.query('High_VIF == True').iloc[0]
        print(f"  {top_vif['Category']}: VIF = {top_vif['VIF']:.2f}")
    
    return {
        'correlation_matrix': corr_matrix,
        'high_correlation_pairs': high_corr_pairs,
        'vif_results': vif_results,
        'summary': {
            'total_categories': len(corr_matrix.columns),
            'high_corr_pairs_count': len(high_corr_pairs),
            'high_vif_categories_count': vif_results['High_VIF'].sum()
        }
    }

def plot_vif_chart(self, vif_results: pd.DataFrame, 
                  top_n: int = 20, vif_threshold: float = 5.0) -> go.Figure:
    """
    VIF値の棒グラフを作成
    
    Args:
        vif_results: VIF計算結果
        top_n: 表示する上位カテゴリー数
        vif_threshold: 警告ライン
        
    Returns:
        plotlyの図オブジェクト
    """
    # 上位N件を取得（NaN除く）
    valid_vif = vif_results.dropna(subset=['VIF']).head(top_n)
    
    # 色の設定（高VIFは赤、それ以外は青）
    colors = ['red' if vif > vif_threshold else 'lightblue' 
             for vif in valid_vif['VIF']]
    
    fig = go.Figure(data=[
        go.Bar(
            x=[name[:20] + '...' if len(name) > 20 else name 
               for name in valid_vif['Category']],
            y=valid_vif['VIF'],
            text=[f'{vif:.1f}' for vif in valid_vif['VIF']],
            textposition='auto',
            marker_color=colors,
            hovertemplate=
            'カテゴリー: %{x}<br>' +
            'VIF: %{y:.2f}<br>' +
            '<extra></extra>'
        )
    ])
    
    # 閾値ライン
    fig.add_hline(y=vif_threshold, line_dash="dash", line_color="red",
                  annotation_text=f"VIF閾値 = {vif_threshold}")
    
    fig.update_layout(
        title=f'VIF値 (Top {top_n})',
        xaxis_title='カテゴリー',
        yaxis_title='VIF値',
        xaxis_tickangle=-45,
        height=600,
        font=dict(size=12)
    )
    
    return fig

# メソッドをクラスに追加
MulticollinearityAnalyzer.generate_multicollinearity_report = generate_multicollinearity_report
MulticollinearityAnalyzer.plot_vif_chart = plot_vif_chart

# 実行用関数の定義
def run_multicollinearity_analysis(results_dict: Dict, 
                                 specific_period: Optional[int] = None,
                                 corr_threshold: float = 0.7,
                                 vif_threshold: float = 5.0,
                                 table_name: str = 'consol_l5') -> Dict:
    """
    マルチコリニアリティ分析の実行
    
    Args:
        results_dict: main()関数の実行結果
        specific_period: 特定期間の指定
        corr_threshold: 高相関の閾値
        vif_threshold: 高VIFの閾値
        table_name: 分析対象テーブル
        
    Returns:
        分析結果辞書
    """
    period_text = f"指定期間: {specific_period}" if specific_period else "最新期間（自動選択）"
    print(f"=== マルチコリニアリティ分析開始（{period_text}） ===\n")
    
    # アナライザーの初期化
    mc_analyzer = MulticollinearityAnalyzer(results_dict['processed_pivot_tables'])
    
    # 1. 総合レポートの生成
    report = mc_analyzer.generate_multicollinearity_report(
        table_name=table_name,
        specific_period=specific_period,
        corr_threshold=corr_threshold,
        vif_threshold=vif_threshold
    )
    
    # 2. 可視化の作成
    print("\n4. 可視化の作成...")
    
    # 相関ヒートマップ
    print("  - 相関ヒートマップ")
    heatmap_fig = mc_analyzer.plot_correlation_heatmap(
        figsize=(12, 10), 
        show_values=False,
        high_corr_threshold=corr_threshold
    )
    heatmap_fig.show()
    
    # VIF棒グラフ
    print("  - VIF棒グラフ")
    vif_fig = mc_analyzer.plot_vif_chart(
        report['vif_results'], 
        top_n=20, 
        vif_threshold=vif_threshold
    )
    vif_fig.show()
    
    # 高相関ネットワーク（ペアが存在する場合のみ）
    if len(report['high_correlation_pairs']) > 0:
        print("  - 高相関ネットワーク")
        network_fig = mc_analyzer.plot_correlation_network(
            threshold=corr_threshold,
            layout='spring'
        )
        if network_fig is not None:
            network_fig.show()
    
    # デンドログラム（カテゴリー数が適切な場合のみ）
    if len(report['correlation_matrix'].columns) <= 50:  # カテゴリー数が多すぎると見づらい
        print("  - 階層クラスタリング")
        dendro_fig = mc_analyzer.plot_dendrogram()
        dendro_fig.show()
    
    # 3. 高相関ペアの詳細表示
    if len(report['high_correlation_pairs']) > 0:
        print(f"\n=== 高相関ペア詳細 (|r| >= {corr_threshold}) ===")
        print(report['high_correlation_pairs'].head(10).to_string(index=False))
    
    # 4. 高VIFカテゴリーの詳細表示
    high_vif_categories = report['vif_results'].query('High_VIF == True')
    if len(high_vif_categories) > 0:
        print(f"\n=== 高VIFカテゴリー詳細 (VIF > {vif_threshold}) ===")
        print(high_vif_categories.head(10)[['Category', 'VIF']].to_string(index=False))
    
    print("\n=== マルチコリニアリティ分析完了 ===")
    
    return {
        'analyzer': mc_analyzer,
        'report': report,
        'figures': {
            'heatmap': heatmap_fig,
            'vif_chart': vif_fig,
            'network': network_fig if len(report['high_correlation_pairs']) > 0 else None,
            'dendrogram': dendro_fig if len(report['correlation_matrix'].columns) <= 50 else None
        }
    }

print("MulticollinearityAnalyzer の完全版と実行関数が定義されました。")


In [ ]:
# マルチコリニアリティ分析クラス（VIF無限大対応版）

# 必要なライブラリの追加インポート
from statsmodels.stats.outliers_influence import variance_inflation_factor
from scipy.stats import pearsonr, spearmanr, kendalltau
from scipy.cluster.hierarchy import linkage, dendrogram
from scipy.linalg import qr
import networkx as nx
from typing import Tuple, Union

class MulticollinearityAnalyzer:
    """
    マルチコリニアリティ分析クラス（VIF無限大対応版）
    """
    
    def __init__(self, pivot_tables: Dict[str, pd.DataFrame]):
        self.pivot_tables = pivot_tables
        self.correlation_matrix = None
        self.current_table = None
        self.current_period = None
        
    def calculate_correlation_matrix(self, table_name: str = 'consol_l5', 
                                   specific_period: Optional[int] = None,
                                   method: str = 'pearson') -> pd.DataFrame:
        """
        相関行列の計算
        
        Args:
            table_name: 分析対象のテーブル名
            specific_period: 特定期間の指定
            method: 相関係数の種類 ('pearson', 'spearman', 'kendall')
            
        Returns:
            相関行列
        """
        if table_name not in self.pivot_tables:
            raise ValueError(f"Table {table_name} not found in pivot_tables")
        
        df = self.pivot_tables[table_name]
        
        # 期間の選択
        if specific_period is not None:
            available_periods = df.index.get_level_values(1).unique()
            if specific_period not in available_periods:
                raise ValueError(f"指定された期間 {specific_period} がデータに存在しません。利用可能な期間: {sorted(available_periods)}")
            target_period = specific_period
            period_df = df.xs(target_period, level=1)
        else:
            target_period = df.index.get_level_values(1).max()
            period_df = df.xs(target_period, level=1)
        
        self.current_table = table_name
        self.current_period = target_period
        
        print(f"相関行列計算: {table_name} (期間: {target_period})")
        print(f"対象カテゴリー数: {len(period_df.columns)}")
        print(f"対象企業数: {len(period_df.index)}")
        
        # 相関行列の計算
        self.correlation_matrix = period_df.corr(method=method)
        
        return self.correlation_matrix
    
    def find_high_correlation_pairs(self, threshold: float = 0.7) -> pd.DataFrame:
        """
        高相関ペアの抽出
        
        Args:
            threshold: 高相関と判定する閾値
            
        Returns:
            高相関ペアのデータフレーム
        """
        if self.correlation_matrix is None:
            raise ValueError("先に calculate_correlation_matrix() を実行してください")
        
        corr_matrix = self.correlation_matrix
        high_corr_pairs = []
        
        # 上三角行列のみ処理（重複避けるため）
        for i in range(len(corr_matrix.columns)):
            for j in range(i + 1, len(corr_matrix.columns)):
                corr_val = corr_matrix.iloc[i, j]
                abs_corr = abs(corr_val)
                
                if abs_corr >= threshold:
                    high_corr_pairs.append({
                        'Category1': corr_matrix.columns[i],
                        'Category2': corr_matrix.columns[j],
                        'Correlation': corr_val,
                        'Abs_Correlation': abs_corr
                    })
        
        pairs_df = pd.DataFrame(high_corr_pairs)
        
        if len(pairs_df) > 0:
            pairs_df = pairs_df.sort_values('Abs_Correlation', ascending=False)
            print(f"高相関ペア数 (|r| >= {threshold}): {len(pairs_df)}")
        else:
            print(f"高相関ペア (|r| >= {threshold}) は見つかりませんでした")
        
        return pairs_df
    
    def calculate_vif(self, table_name: str = 'consol_l5', 
                     specific_period: Optional[int] = None,
                     vif_threshold: float = 5.0) -> pd.DataFrame:
        """
        VIF（分散拡大係数）の計算（従来版）
        
        Args:
            table_name: 分析対象のテーブル名
            specific_period: 特定期間の指定
            vif_threshold: 問題ありとするVIFの閾値
            
        Returns:
            VIF結果のデータフレーム
        """
        if table_name not in self.pivot_tables:
            raise ValueError(f"Table {table_name} not found")
        
        df = self.pivot_tables[table_name]
        
        # 期間の選択
        if specific_period is not None:
            target_period = specific_period
            period_df = df.xs(target_period, level=1)
        else:
            target_period = df.index.get_level_values(1).max()
            period_df = df.xs(target_period, level=1)
        
        print(f"VIF計算開始 (期間: {target_period})")
        print(f"対象カテゴリー数: {len(period_df.columns)}")
        
        # データクリーニング
        clean_df = period_df.fillna(0)
        clean_df = clean_df.replace([np.inf, -np.inf], 0)
        
        vif_data = []
        
        # 分散がゼロのカラムをチェック
        zero_var_cols = clean_df.var() == 0
        if zero_var_cols.sum() > 0:
            print(f"警告: 分散がゼロのカテゴリーが {zero_var_cols.sum()} 個あります")
        
        for i, col in enumerate(clean_df.columns):
            try:
                if clean_df[col].var() == 0:
                    vif_value = 0.0
                else:
                    vif_value = variance_inflation_factor(clean_df.values, i)
                
                vif_data.append({
                    'Category': col,
                    'VIF': vif_value,
                    'High_VIF': vif_value > vif_threshold if not np.isinf(vif_value) else True
                })
                
            except Exception as e:
                print(f"VIF計算エラー ({col}): {e}")
                vif_data.append({
                    'Category': col,
                    'VIF': np.nan,
                    'High_VIF': False
                })
        
        vif_df = pd.DataFrame(vif_data)
        
        # 結果のサマリー
        high_vif_count = vif_df['High_VIF'].sum()
        inf_vif_count = np.isinf(vif_df['VIF']).sum()
        
        print(f"VIF > {vif_threshold}: {high_vif_count} 個")
        print(f"VIF = ∞: {inf_vif_count} 個")
        
        return vif_df.sort_values('VIF', ascending=False, na_position='last')
    
    def plot_correlation_heatmap(self, figsize: Tuple[int, int] = (12, 10), 
                               show_values: bool = False,
                               high_corr_threshold: float = 0.7) -> go.Figure:
        """
        相関ヒートマップの作成
        """
        if self.correlation_matrix is None:
            raise ValueError("先に calculate_correlation_matrix() を実行してください")
        
        corr_matrix = self.correlation_matrix
        
        # カテゴリー名を短縮
        short_names = [name[:25] + '...' if len(name) > 25 else name for name in corr_matrix.columns]
        
        fig = go.Figure(data=go.Heatmap(
            z=corr_matrix.values,
            x=short_names,
            y=short_names,
            colorscale='RdBu_r',
            zmin=-1,
            zmax=1,
            colorbar=dict(title="相関係数"),
            hoverongaps=False,
            hovertemplate=
            'カテゴリー1: %{y}<br>' +
            'カテゴリー2: %{x}<br>' +
            '相関係数: %{z:.3f}<br>' +
            '<extra></extra>'
        ))
        
        fig.update_layout(
            title=f'カテゴリー間相関行列 ({self.current_table}, 期間: {self.current_period})',
            xaxis_title='カテゴリー',
            yaxis_title='カテゴリー',
            width=figsize[0] * 100,
            height=figsize[1] * 100,
            font=dict(size=10)
        )
        
        return fig
    
    def plot_correlation_network(self, threshold: float = 0.7, 
                               layout: str = 'spring') -> go.Figure:
        """
        高相関カテゴリーのネットワーク図
        """
        if self.correlation_matrix is None:
            raise ValueError("先に calculate_correlation_matrix() を実行してください")
        
        # ネットワークグラフの作成
        G = nx.Graph()
        corr_matrix = self.correlation_matrix
        
        # ノードの追加
        for col in corr_matrix.columns:
            G.add_node(col)
        
        # エッジの追加（高相関のペア）
        for i in range(len(corr_matrix.columns)):
            for j in range(i + 1, len(corr_matrix.columns)):
                corr_val = abs(corr_matrix.iloc[i, j])
                if corr_val >= threshold:
                    G.add_edge(corr_matrix.columns[i], corr_matrix.columns[j], 
                             weight=corr_val)
        
        if len(G.edges()) == 0:
            print(f"閾値 {threshold} 以上の相関を持つペアが見つかりません")
            return None
        
        # レイアウトの計算
        if layout == 'spring':
            pos = nx.spring_layout(G, k=1, iterations=50)
        elif layout == 'circular':
            pos = nx.circular_layout(G)
        else:
            pos = nx.random_layout(G)
        
        # プロットの作成
        edge_x, edge_y = [], []
        for edge in G.edges():
            x0, y0 = pos[edge[0]]
            x1, y1 = pos[edge[1]]
            edge_x.extend([x0, x1, None])
            edge_y.extend([y0, y1, None])
        
        node_x = [pos[node][0] for node in G.nodes()]
        node_y = [pos[node][1] for node in G.nodes()]
        node_names = [name[:20] + '...' if len(name) > 20 else name for name in G.nodes()]
        
        fig = go.Figure()
        
        # エッジ
        fig.add_trace(go.Scatter(x=edge_x, y=edge_y,
                               line=dict(width=1, color='gray'),
                               hoverinfo='none',
                               mode='lines'))
        
        # ノード
        fig.add_trace(go.Scatter(x=node_x, y=node_y,
                               mode='markers+text',
                               marker=dict(size=15, color='lightblue',
                                         line=dict(width=1, color='black')),
                               text=node_names,
                               textposition='middle center',
                               hovertemplate='カテゴリー: %{text}<extra></extra>'))
        
        fig.update_layout(
            title=f'高相関ネットワーク (相関 >= {threshold})',
            showlegend=False,
            width=800,
            height=600,
            xaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
            yaxis=dict(showgrid=False, zeroline=False, showticklabels=False)
        )
        
        return fig

print("MulticollinearityAnalyzerクラスが定義されました。")


In [ ]:
# マルチコリニアリティ分析のデモ実行関数と使用例

def demo_multicollinearity_analysis(specific_period: Optional[int] = None,
                                   corr_threshold: float = 0.7,
                                   vif_threshold: float = 5.0):
    """
    マルチコリニアリティ分析のデモ実行
    
    Args:
        specific_period: 特定期間の指定（例：202403）
        corr_threshold: 高相関の閾値（デフォルト：0.7）
        vif_threshold: 高VIFの閾値（デフォルト：5.0）
    """
    period_text = f"指定期間: {specific_period}" if specific_period else "最新期間（自動選択）"
    print(f"=== マルチコリニアリティ分析デモ（{period_text}） ===\n")
    
    # 1. メインデータ処理の実行
    print("1. メインデータ処理の実行...")
    results = main()
    
    # 2. マルチコリニアリティ分析の実行
    print("\n2. マルチコリニアリティ分析の実行...")
    mc_results = run_multicollinearity_analysis(
        results_dict=results,
        specific_period=specific_period,
        corr_threshold=corr_threshold,
        vif_threshold=vif_threshold
    )
    
    # 3. 分析結果の詳細表示
    print("\n3. 分析結果の詳細...")
    report = mc_results['report']
    
    # 高相関ペアの詳細分析
    if len(report['high_correlation_pairs']) > 0:
        print(f"\n--- 高相関ペア分析 (閾値: {corr_threshold}) ---")
        top_5_pairs = report['high_correlation_pairs'].head(5)
        for idx, row in top_5_pairs.iterrows():
            print(f"  {row['Category1'][:30]}... ↔ {row['Category2'][:30]}... : r = {row['Correlation']:.3f}")
    
    # VIF分析
    high_vif = report['vif_results'].query('High_VIF == True')
    if len(high_vif) > 0:
        print(f"\n--- 高VIF分析 (閾値: {vif_threshold}) ---")
        top_5_vif = high_vif.head(5)
        for idx, row in top_5_vif.iterrows():
            print(f"  {row['Category'][:40]}... : VIF = {row['VIF']:.2f}")
    
    # 4. 回帰分析への推奨事項
    print("\n4. 回帰分析への推奨事項...")
    print_regression_recommendations(report, corr_threshold, vif_threshold)
    
    print("\n=== デモ完了 ===")
    return results, mc_results

def print_regression_recommendations(report: Dict, corr_threshold: float, vif_threshold: float):
    """
    回帰分析への推奨事項を表示
    
    Args:
        report: 分析レポート
        corr_threshold: 相関閾値
        vif_threshold: VIF閾値
    """
    high_corr_count = len(report['high_correlation_pairs'])
    high_vif_count = report['vif_results']['High_VIF'].sum()
    
    print("【回帰分析への影響と対策】")
    
    if high_corr_count == 0 and high_vif_count == 0:
        print("✅ マルチコリニアリティの問題は検出されませんでした。")
        print("   すべてのカテゴリーを回帰分析に使用できます。")
    else:
        if high_corr_count > 0:
            print(f"⚠️  高相関ペア: {high_corr_count} 組")
            print("   対策: 高相関のカテゴリーペアのうち、片方を除外することを検討してください。")
        
        if high_vif_count > 0:
            print(f"⚠️  高VIFカテゴリー: {high_vif_count} 個")
            print("   対策: VIF > 10 のカテゴリーは除外、VIF 5-10 のカテゴリーは注意深く使用してください。")
        
        # 除外候補の提案
        if high_vif_count > 0:
            print(f"\n【除外候補カテゴリー（VIF > {vif_threshold}）】")
            exclude_candidates = report['vif_results'].query('High_VIF == True').head(5)
            for idx, row in exclude_candidates.iterrows():
                print(f"  - {row['Category'][:50]}... (VIF: {row['VIF']:.2f})")

def quick_correlation_check(results_dict: Dict, 
                          category1: str, 
                          category2: str,
                          specific_period: Optional[int] = None,
                          table_name: str = 'consol_l5') -> float:
    """
    特定の2つのカテゴリー間の相関を素早くチェック
    
    Args:
        results_dict: main()の実行結果
        category1: カテゴリー1
        category2: カテゴリー2
        specific_period: 特定期間
        table_name: テーブル名
        
    Returns:
        相関係数
    """
    analyzer = MulticollinearityAnalyzer(results_dict['processed_pivot_tables'])
    corr_matrix = analyzer.calculate_correlation_matrix(table_name, specific_period, method='pearson')
    
    if category1 in corr_matrix.columns and category2 in corr_matrix.columns:
        correlation = corr_matrix.loc[category1, category2]
        print(f"相関係数 ({category1} ↔ {category2}): {correlation:.3f}")
        
        if abs(correlation) >= 0.8:
            print("⚠️  非常に高い相関です。回帰分析では片方を除外することを強く推奨します。")
        elif abs(correlation) >= 0.7:
            print("⚠️  高い相関です。回帰分析での使用時は注意が必要です。")
        elif abs(correlation) >= 0.5:
            print("ℹ️  中程度の相関です。")
        else:
            print("✅ 低い相関です。")
        
        return correlation
    else:
        print("指定されたカテゴリーが見つかりません。")
        return None

# 実行方法の説明
print("""
=== マルチコリニアリティ分析の実行方法 ===

【基本的な実行】
1. 完全デモ実行（最新期間、デフォルト閾値）:
   results, mc_results = demo_multicollinearity_analysis()

2. 特定期間での実行:
   results, mc_results = demo_multicollinearity_analysis(specific_period=202403)

3. 閾値をカスタマイズ:
   results, mc_results = demo_multicollinearity_analysis(
       corr_threshold=0.8,    # 相関閾値を0.8に
       vif_threshold=10.0     # VIF閾値を10に
   )

【個別実行】
1. データ準備:
   results = main()

2. マルチコリニアリティ分析:
   mc_results = run_multicollinearity_analysis(results)

3. 特定カテゴリーペアの相関チェック:
   correlation = quick_correlation_check(results, 'Technology', 'Software')

【分析結果の活用】
- mc_results['report']['high_correlation_pairs']: 高相関ペア一覧
- mc_results['report']['vif_results']: VIF計算結果
- mc_results['figures']: 各種可視化グラフ

【閾値の目安】
- 相関係数: |r| >= 0.7 で注意、|r| >= 0.8 で除外検討
- VIF: VIF > 5 で注意、VIF > 10 で除外検討
""")


In [ ]:
# VIF無限大の対処とマルチコリニアリティ診断の改良版

def calculate_vif_improved(self, table_name: str = 'consol_l5', 
                         specific_period: Optional[int] = None,
                         vif_threshold: float = 5.0,
                         handle_perfect_multicollinearity: bool = True) -> pd.DataFrame:
    """
    改良版VIF計算（完全共線性の検出・対処機能付き）
    
    Args:
        table_name: 分析対象のテーブル名
        specific_period: 特定期間の指定
        vif_threshold: 問題ありとするVIFの閾値
        handle_perfect_multicollinearity: 完全共線性を自動処理するか
        
    Returns:
        VIF結果のデータフレーム
    """
    if table_name not in self.pivot_tables:
        raise ValueError(f"Table {table_name} not found")
    
    df = self.pivot_tables[table_name]
    
    # 期間の選択
    if specific_period is not None:
        target_period = specific_period
        period_df = df.xs(target_period, level=1)
    else:
        target_period = df.index.get_level_values(1).max()
        period_df = df.xs(target_period, level=1)
    
    print(f"VIF計算開始 (期間: {target_period})")
    print(f"元のカテゴリー数: {len(period_df.columns)}")
    
    # データクリーニング
    clean_df = period_df.fillna(0)
    clean_df = clean_df.replace([np.inf, -np.inf], 0)
    
    # 1. 分散が0のカラムを除去
    zero_var_cols = clean_df.var() <= 1e-10
    if zero_var_cols.sum() > 0:
        print(f"分散ゼロのカテゴリーを除去: {zero_var_cols.sum()} 個")
        print("除去されたカテゴリー:")
        for col in clean_df.columns[zero_var_cols]:
            print(f"  - {col}")
    
    clean_df = clean_df.loc[:, ~zero_var_cols]
    
    # 2. 完全相関の検出
    if handle_perfect_multicollinearity:
        clean_df, removed_perfect_corr = self._remove_perfect_correlations(clean_df)
    else:
        removed_perfect_corr = []
    
    # 3. 線形従属性の検出
    if handle_perfect_multicollinearity:
        clean_df, removed_linear_dep = self._remove_linear_dependence(clean_df)
    else:
        removed_linear_dep = []
    
    print(f"VIF計算対象カテゴリー数: {len(clean_df.columns)}")
    
    # 4. VIF計算
    vif_data = []
    
    # 除去されたカテゴリーの記録
    for col in period_df.columns[zero_var_cols]:
        vif_data.append({
            'Category': col,
            'VIF': 0.0,
            'High_VIF': False,
            'Issue': 'Zero_Variance',
            'Description': '分散がゼロ'
        })
    
    for col in removed_perfect_corr:
        vif_data.append({
            'Category': col,
            'VIF': np.inf,
            'High_VIF': True,
            'Issue': 'Perfect_Correlation',
            'Description': '完全相関により除去'
        })
    
    for col in removed_linear_dep:
        vif_data.append({
            'Category': col,
            'VIF': np.inf,
            'High_VIF': True,
            'Issue': 'Linear_Dependence',
            'Description': '線形従属により除去'
        })
    
    # 残ったカテゴリーのVIF計算
    if len(clean_df.columns) >= 2:
        for i, col in enumerate(clean_df.columns):
            try:
                vif_value = variance_inflation_factor(clean_df.values, i)
                
                # 無限大や異常値のチェック
                if np.isinf(vif_value) or np.isnan(vif_value) or vif_value > 1000:
                    vif_data.append({
                        'Category': col,
                        'VIF': np.inf,
                        'High_VIF': True,
                        'Issue': 'Computational_Issue',
                        'Description': 'VIF計算で無限大'
                    })
                else:
                    vif_data.append({
                        'Category': col,
                        'VIF': vif_value,
                        'High_VIF': vif_value > vif_threshold,
                        'Issue': 'Normal' if vif_value <= vif_threshold else 'High_VIF',
                        'Description': f'VIF = {vif_value:.2f}'
                    })
                    
            except Exception as e:
                print(f"VIF計算エラー ({col}): {e}")
                vif_data.append({
                    'Category': col,
                    'VIF': np.nan,
                    'High_VIF': False,
                    'Issue': 'Calculation_Error',
                    'Description': f'計算エラー: {str(e)}'
                })
    
    vif_df = pd.DataFrame(vif_data)
    
    # 結果のサマリー
    total_cats = len(vif_df)
    high_vif_count = vif_df['High_VIF'].sum()
    inf_vif_count = np.isinf(vif_df['VIF']).sum()
    
    print(f"\n=== VIF計算結果サマリー ===")
    print(f"総カテゴリー数: {total_cats}")
    print(f"高VIF（> {vif_threshold}）: {high_vif_count}")
    print(f"無限大VIF: {inf_vif_count}")
    
    if inf_vif_count > 0:
        print(f"\n無限大VIFの内訳:")
        inf_issues = vif_df[np.isinf(vif_df['VIF'])]['Issue'].value_counts()
        for issue, count in inf_issues.items():
            print(f"  - {issue}: {count} 個")
    
    return vif_df.sort_values('VIF', ascending=False, na_position='last')

def _remove_perfect_correlations(self, df: pd.DataFrame, threshold: float = 0.999) -> Tuple[pd.DataFrame, List[str]]:
    """
    完全相関（または準完全相関）のカラムを除去
    
    Args:
        df: データフレーム
        threshold: 完全相関とみなす閾値
        
    Returns:
        (クリーニング後のデータフレーム, 除去されたカラムのリスト)
    """
    removed_cols = []
    corr_matrix = df.corr()
    
    # 上三角行列で完全相関をチェック
    for i in range(len(corr_matrix.columns)):
        for j in range(i + 1, len(corr_matrix.columns)):
            corr_val = abs(corr_matrix.iloc[i, j])
            if corr_val >= threshold:
                col_to_remove = corr_matrix.columns[j]  # 後の方のカラムを除去
                if col_to_remove not in removed_cols:
                    removed_cols.append(col_to_remove)
                    print(f"完全相関により除去: {col_to_remove} (相関係数: {corr_val:.4f})")
    
    # 除去
    remaining_df = df.drop(columns=removed_cols)
    return remaining_df, removed_cols

def _remove_linear_dependence(self, df: pd.DataFrame, tolerance: float = 1e-10) -> Tuple[pd.DataFrame, List[str]]:
    """
    線形従属のカラムを除去
    
    Args:
        df: データフレーム
        tolerance: 線形従属と判定する許容誤差
        
    Returns:
        (クリーニング後のデータフレーム, 除去されたカラムのリスト)
    """
    from scipy.linalg import qr
    
    removed_cols = []
    
    try:
        # QR分解で線形独立性をチェック
        Q, R = qr(df.values)
        
        # 対角要素が小さいものは線形従属
        diag = np.abs(np.diag(R))
        dependent_indices = np.where(diag < tolerance)[0]
        
        if len(dependent_indices) > 0:
            for idx in dependent_indices:
                if idx < len(df.columns):
                    col_to_remove = df.columns[idx]
                    removed_cols.append(col_to_remove)
                    print(f"線形従属により除去: {col_to_remove}")
        
        remaining_df = df.drop(columns=removed_cols)
        return remaining_df, removed_cols
        
    except Exception as e:
        print(f"線形従属性チェックでエラー: {e}")
        return df, []

# メソッドをクラスに追加
MulticollinearityAnalyzer.calculate_vif_improved = calculate_vif_improved
MulticollinearityAnalyzer._remove_perfect_correlations = _remove_perfect_correlations
MulticollinearityAnalyzer._remove_linear_dependence = _remove_linear_dependence

print("VIF無限大対処機能が追加されました。")


In [ ]:
# VIF無限大の診断・説明機能と改良版実行関数

def diagnose_infinite_vif(self, vif_results: pd.DataFrame) -> Dict:
    """
    VIF無限大の原因を診断・説明
    
    Args:
        vif_results: VIF計算結果
        
    Returns:
        診断結果の辞書
    """
    inf_vif_df = vif_results[np.isinf(vif_results['VIF'])]
    
    diagnosis = {
        'total_infinite_vif': len(inf_vif_df),
        'causes': {},
        'recommendations': []
    }
    
    if len(inf_vif_df) == 0:
        diagnosis['summary'] = "VIF無限大のカテゴリーはありません。"
        return diagnosis
    
    # 原因別の集計
    cause_counts = inf_vif_df['Issue'].value_counts().to_dict()
    diagnosis['causes'] = cause_counts
    
    print("=== VIF無限大の診断結果 ===\n")
    
    # 各原因の説明
    explanations = {
        'Perfect_Correlation': {
            'title': '完全相関（Perfect Correlation）',
            'explanation': 'カテゴリー間の相関係数が1.0または-1.0に近い状態です。',
            'mathematical': 'VIF = 1/(1-R²) で、R²=1のときVIF→∞',
            'causes': [
                '同じ産業の細分化カテゴリー（例：Technology と Software）',
                '地理的な重複（例：North America と United States）',
                '概念的な包含関係（例：Manufacturing と Automotive）'
            ],
            'solution': '高相関ペアのうち片方を除外する'
        },
        'Linear_Dependence': {
            'title': '線形従属（Linear Dependence）',
            'explanation': '複数のカテゴリーの線形結合で表現できるカテゴリーが存在します。',
            'mathematical': '例：Category_A + Category_B = Category_C の関係',
            'causes': [
                '階層的なカテゴリー構造（親カテゴリー = 子カテゴリーの合計）',
                '相互排他的でないカテゴリーの組み合わせ',
                'データの集計方法による重複'
            ],
            'solution': '線形従属のカテゴリーを除外する'
        },
        'Zero_Variance': {
            'title': '分散ゼロ（Zero Variance）',
            'explanation': 'すべての企業で同じ値を持つカテゴリーです。',
            'mathematical': 'Var(X) = 0 → 回帰分析で使用不可',
            'causes': [
                'すべての企業が該当しないカテゴリー',
                'データの期間や地域制限による',
                '非常に特殊なニッチカテゴリー'
            ],
            'solution': 'カテゴリーを除外する（分析に寄与しない）'
        },
        'Computational_Issue': {
            'title': '計算上の問題（Computational Issue）',
            'explanation': 'VIF計算過程で数値的な問題が発生しました。',
            'mathematical': '行列の特異性や数値精度の問題',
            'causes': [
                '非常に高い相関による数値的不安定性',
                '行列の条件数が大きすぎる',
                'データの精度やスケールの問題'
            ],
            'solution': 'データの前処理や変数選択を見直す'
        }
    }
    
    # 診断結果の詳細表示
    for issue, count in cause_counts.items():
        if issue in explanations:
            exp = explanations[issue]
            print(f"🔍 **{exp['title']}** ({count} 件)")
            print(f"   {exp['explanation']}")
            print(f"   数学的説明: {exp['mathematical']}")
            print(f"   主な原因:")
            for cause in exp['causes']:
                print(f"     - {cause}")
            print(f"   対処法: {exp['solution']}")
            print()
            
            diagnosis['recommendations'].append({
                'issue': issue,
                'count': count,
                'solution': exp['solution'],
                'categories': inf_vif_df[inf_vif_df['Issue'] == issue]['Category'].tolist()
            })
    
    # 総合的な推奨事項
    print("=== 総合的な推奨事項 ===")
    if diagnosis['total_infinite_vif'] > 0:
        print("⚠️  VIF無限大のカテゴリーが検出されました。")
        print("📋 推奨対応:")
        print("   1. 完全相関・線形従属のカテゴリーペアを特定")
        print("   2. ビジネス的に重要なカテゴリーを優先して保持")
        print("   3. 除外後に再度VIF計算を実行")
        print("   4. VIF < 10 を目標に変数選択を調整")
    
    diagnosis['summary'] = f"VIF無限大: {diagnosis['total_infinite_vif']} 件"
    
    return diagnosis

def run_multicollinearity_analysis_improved(results_dict: Dict, 
                                           specific_period: Optional[int] = None,
                                           corr_threshold: float = 0.7,
                                           vif_threshold: float = 5.0,
                                           table_name: str = 'consol_l5',
                                           use_improved_vif: bool = True) -> Dict:
    """
    改良版マルチコリニアリティ分析の実行
    
    Args:
        results_dict: main()関数の実行結果
        specific_period: 特定期間の指定
        corr_threshold: 高相関の閾値
        vif_threshold: 高VIFの閾値
        table_name: 分析対象テーブル
        use_improved_vif: 改良版VIF計算を使用するか
        
    Returns:
        分析結果辞書
    """
    period_text = f"指定期間: {specific_period}" if specific_period else "最新期間（自動選択）"
    print(f"=== 改良版マルチコリニアリティ分析開始（{period_text}） ===\n")
    
    # アナライザーの初期化
    mc_analyzer = MulticollinearityAnalyzer(results_dict['processed_pivot_tables'])
    
    # 1. 相関行列の計算
    print("1. 相関行列の計算...")
    corr_matrix = mc_analyzer.calculate_correlation_matrix(table_name, specific_period)
    
    # 2. 高相関ペアの抽出
    print("\n2. 高相関ペアの抽出...")
    high_corr_pairs = mc_analyzer.find_high_correlation_pairs(corr_threshold)
    
    # 3. 改良版VIF計算
    print("\n3. VIF計算...")
    if use_improved_vif:
        vif_results = mc_analyzer.calculate_vif_improved(
            table_name, specific_period, vif_threshold, handle_perfect_multicollinearity=True
        )
    else:
        vif_results = mc_analyzer.calculate_vif(
            table_name, specific_period, vif_threshold
        )
    
    # 4. VIF無限大の診断
    print("\n4. VIF無限大の診断...")
    vif_diagnosis = mc_analyzer.diagnose_infinite_vif(vif_results)
    
    # 5. 結果のサマリー
    print(f"\n=== サマリー ===")
    print(f"総カテゴリー数: {len(corr_matrix.columns)}")
    print(f"高相関ペア数 (|r| >= {corr_threshold}): {len(high_corr_pairs)}")
    print(f"高VIFカテゴリー数 (VIF > {vif_threshold}): {vif_results['High_VIF'].sum()}")
    print(f"VIF無限大カテゴリー数: {vif_diagnosis['total_infinite_vif']}")
    
    # 6. 可視化の作成
    print("\n5. 可視化の作成...")
    
    # 相関ヒートマップ
    print("  - 相関ヒートマップ")
    heatmap_fig = mc_analyzer.plot_correlation_heatmap(
        figsize=(12, 10), 
        show_values=False,
        high_corr_threshold=corr_threshold
    )
    heatmap_fig.show()
    
    # 改良版VIF棒グラフ（無限大の色分け付き）
    print("  - VIF棒グラフ（改良版）")
    vif_fig = plot_vif_chart_improved(vif_results, top_n=25, vif_threshold=vif_threshold)
    vif_fig.show()
    
    # 高相関ネットワーク（ペアが存在する場合のみ）
    if len(high_corr_pairs) > 0:
        print("  - 高相関ネットワーク")
        network_fig = mc_analyzer.plot_correlation_network(
            threshold=corr_threshold,
            layout='spring'
        )
        if network_fig is not None:
            network_fig.show()
    
    # 7. 詳細結果の表示
    if len(high_corr_pairs) > 0:
        print(f"\n=== 高相関ペア詳細 (|r| >= {corr_threshold}) ===")
        print(high_corr_pairs.head(10).to_string(index=False))
    
    # VIF結果の表示（問題があるもののみ）
    problem_vif = vif_results[
        (vif_results['High_VIF'] == True) | 
        (vif_results['Issue'] != 'Normal')
    ]
    if len(problem_vif) > 0:
        print(f"\n=== 問題のあるVIFカテゴリー詳細 ===")
        display_cols = ['Category', 'VIF', 'Issue', 'Description']
        print(problem_vif[display_cols].head(15).to_string(index=False))
    
    print("\n=== 改良版マルチコリニアリティ分析完了 ===")
    
    return {
        'analyzer': mc_analyzer,
        'correlation_matrix': corr_matrix,
        'high_correlation_pairs': high_corr_pairs,
        'vif_results': vif_results,
        'vif_diagnosis': vif_diagnosis,
        'figures': {
            'heatmap': heatmap_fig,
            'vif_chart': vif_fig,
            'network': network_fig if len(high_corr_pairs) > 0 else None
        },
        'summary': {
            'total_categories': len(corr_matrix.columns),
            'high_corr_pairs_count': len(high_corr_pairs),
            'high_vif_categories_count': vif_results['High_VIF'].sum(),
            'infinite_vif_count': vif_diagnosis['total_infinite_vif']
        }
    }

def plot_vif_chart_improved(vif_results: pd.DataFrame, 
                          top_n: int = 25, 
                          vif_threshold: float = 5.0) -> go.Figure:
    """
    改良版VIF棒グラフ（無限大の色分け付き）
    """
    # 無限大を1000に変換して表示用に調整
    display_vif = vif_results.copy()
    display_vif['VIF_Display'] = display_vif['VIF'].replace([np.inf], 1000)
    
    # 上位N件を取得（NaN除く）
    valid_vif = display_vif.dropna(subset=['VIF_Display']).head(top_n)
    
    # 色の設定（問題別に色分け）
    def get_color(row):
        if row['Issue'] == 'Perfect_Correlation':
            return 'red'
        elif row['Issue'] == 'Linear_Dependence':
            return 'darkred'
        elif row['Issue'] == 'Computational_Issue':
            return 'orange'
        elif row['Issue'] == 'Zero_Variance':
            return 'gray'
        elif row['VIF'] > vif_threshold:
            return 'coral'
        else:
            return 'lightblue'
    
    colors = [get_color(row) for _, row in valid_vif.iterrows()]
    
    fig = go.Figure(data=[
        go.Bar(
            x=[name[:20] + '...' if len(name) > 20 else name 
               for name in valid_vif['Category']],
            y=valid_vif['VIF_Display'],
            text=[f'{vif:.1f}' if not np.isinf(vif) else '∞' 
                  for vif in valid_vif['VIF']],
            textposition='auto',
            marker_color=colors,
            hovertemplate=
            'カテゴリー: %{x}<br>' +
            'VIF: %{text}<br>' +
            '問題: ' + valid_vif['Issue'].astype(str) + '<br>' +
            '<extra></extra>'
        )
    ])
    
    # 閾値ライン
    fig.add_hline(y=vif_threshold, line_dash="dash", line_color="red",
                  annotation_text=f"VIF閾値 = {vif_threshold}")
    
    fig.update_layout(
        title=f'VIF値（改良版・色分け付き） (Top {top_n})',
        xaxis_title='カテゴリー',
        yaxis_title='VIF値',
        xaxis_tickangle=-45,
        height=600,
        font=dict(size=12),
        annotations=[
            dict(
                text="色の意味: 赤=完全相関, 暗赤=線形従属, オレンジ=計算問題, 灰=分散ゼロ, サンゴ=高VIF, 青=正常",
                showarrow=False,
                xref='paper', yref='paper',
                x=0.5, y=-0.15,
                xanchor='center', yanchor='top',
                font=dict(size=10)
            )
        ]
    )
    
    return fig

# メソッドをクラスに追加
MulticollinearityAnalyzer.diagnose_infinite_vif = diagnose_infinite_vif

print("VIF無限大診断機能と改良版実行関数が追加されました。")


In [ ]:
# VIF無限大対応版のクイックスタートガイド

print("""
=== VIF無限大問題解決のためのクイックスタートガイド ===

【簡単実行（推奨）】
>>> results, mc_results = demo_multicollinearity_analysis_improved()

【段階的実行】
1. データ準備:
   >>> results = main()

2. 改良版マルチコリニアリティ分析:
   >>> mc_results = run_multicollinearity_analysis_improved(results)

3. VIF無限大の詳細分析:
   >>> vif_diagnosis = mc_results['vif_diagnosis']
   >>> print(f"無限大VIF: {vif_diagnosis['total_infinite_vif']} 件")

【特定期間での分析】
>>> results, mc_results = demo_multicollinearity_analysis_improved(specific_period=202403)

【VIF無限大の詳細説明】
>>> explain_vif_infinity()

【結果の確認】
- 相関行列: mc_results['correlation_matrix']
- 高相関ペア: mc_results['high_correlation_pairs'] 
- VIF結果: mc_results['vif_results']
- VIF診断: mc_results['vif_diagnosis']

【問題カテゴリーの確認】
>>> problem_vif = mc_results['vif_results'][mc_results['vif_results']['High_VIF']]
>>> print(problem_vif[['Category', 'VIF', 'Issue']].head())

【回帰分析戦略の確認】
>>> generate_regression_strategy(mc_results)

=== 実行準備完了 ===
上記のコマンドをコピー&ペーストして実行してください！
""")

# テスト実行用の簡易関数
def test_vif_analysis():
    """VIF無限大機能のテスト実行"""
    print("VIF無限大対応機能のテストを実行します...")
    print("実際の分析を行うには demo_multicollinearity_analysis_improved() を実行してください。")
    return True

print("VIF無限大対応版の準備が完了しました！")


In [ ]:
# 改良版マルチコリニアリティ分析のデモ実行関数と説明

def demo_multicollinearity_analysis_improved(specific_period: Optional[int] = None,
                                            corr_threshold: float = 0.7,
                                            vif_threshold: float = 5.0):
    """
    改良版マルチコリニアリティ分析のデモ実行（VIF無限大対応版）
    
    Args:
        specific_period: 特定期間の指定（例：202403）
        corr_threshold: 高相関の閾値（デフォルト：0.7）
        vif_threshold: 高VIFの閾値（デフォルト：5.0）
    """
    period_text = f"指定期間: {specific_period}" if specific_period else "最新期間（自動選択）"
    print(f"=== 改良版マルチコリニアリティ分析デモ（{period_text}） ===\n")
    
    # 1. メインデータ処理の実行
    print("1. メインデータ処理の実行...")
    results = main()
    
    # 2. 改良版マルチコリニアリティ分析の実行
    print("\n2. 改良版マルチコリニアリティ分析の実行...")
    mc_results = run_multicollinearity_analysis_improved(
        results_dict=results,
        specific_period=specific_period,
        corr_threshold=corr_threshold,
        vif_threshold=vif_threshold,
        use_improved_vif=True  # 改良版VIF計算を使用
    )
    
    # 3. VIF無限大の詳細分析
    vif_diagnosis = mc_results['vif_diagnosis']
    if vif_diagnosis['total_infinite_vif'] > 0:
        print(f"\n3. VIF無限大の詳細分析結果...")
        print(f"無限大VIF: {vif_diagnosis['total_infinite_vif']} 件")
        
        # 原因別の内訳
        for issue, count in vif_diagnosis['causes'].items():
            print(f"  - {issue}: {count} 件")
        
        # 具体的な推奨除外カテゴリー
        print(f"\n推奨除外カテゴリー:")
        for rec in vif_diagnosis['recommendations']:
            if rec['count'] > 0:
                print(f"  【{rec['issue']}】({rec['count']} 件)")
                for i, cat in enumerate(rec['categories'][:3]):  # 上位3件のみ表示
                    print(f"    {i+1}. {cat[:50]}...")
                if len(rec['categories']) > 3:
                    print(f"    ... 他 {len(rec['categories']) - 3} 件")
    
    # 4. 回帰分析への具体的推奨事項
    print(f"\n4. 回帰分析への具体的推奨事項...")
    generate_regression_strategy(mc_results)
    
    print("\n=== 改良版デモ完了 ===")
    return results, mc_results

def generate_regression_strategy(mc_results: Dict):
    """
    回帰分析のための具体的な戦略を生成
    
    Args:
        mc_results: マルチコリニアリティ分析結果
    """
    vif_results = mc_results['vif_results']
    high_corr_pairs = mc_results['high_correlation_pairs']
    vif_diagnosis = mc_results['vif_diagnosis']
    
    print("【回帰分析のための変数選択戦略】")
    
    # 1. 除外すべきカテゴリーの特定
    must_exclude = vif_results[
        (np.isinf(vif_results['VIF'])) | 
        (vif_results['VIF'] > 50)
    ]['Category'].tolist()
    
    should_exclude = vif_results[
        (vif_results['VIF'] > 10) & (vif_results['VIF'] <= 50)
    ]['Category'].tolist()
    
    consider_exclude = vif_results[
        (vif_results['VIF'] > 5) & (vif_results['VIF'] <= 10)
    ]['Category'].tolist()
    
    print(f"\n🚫 必須除外 (VIF > 50 or ∞): {len(must_exclude)} 件")
    if len(must_exclude) > 0:
        for cat in must_exclude[:5]:
            print(f"  - {cat[:60]}...")
        if len(must_exclude) > 5:
            print(f"  ... 他 {len(must_exclude) - 5} 件")
    
    print(f"\n⚠️ 除外推奨 (10 < VIF ≤ 50): {len(should_exclude)} 件")
    if len(should_exclude) > 0:
        for cat in should_exclude[:5]:
            print(f"  - {cat[:60]}...")
        if len(should_exclude) > 5:
            print(f"  ... 他 {len(should_exclude) - 5} 件")
    
    print(f"\n🤔 除外検討 (5 < VIF ≤ 10): {len(consider_exclude)} 件")
    if len(consider_exclude) > 0:
        for cat in consider_exclude[:3]:
            print(f"  - {cat[:60]}...")
        if len(consider_exclude) > 3:
            print(f"  ... 他 {len(consider_exclude) - 3} 件")
    
    # 2. 使用可能なカテゴリーの数
    usable_categories = vif_results[
        (vif_results['VIF'] <= 5) & 
        (~np.isinf(vif_results['VIF'])) & 
        (~np.isnan(vif_results['VIF']))
    ]
    
    print(f"\n✅ 使用可能 (VIF ≤ 5): {len(usable_categories)} 件")
    
    # 3. 段階的な変数選択の推奨
    print(f"\n【段階的変数選択の推奨手順】")
    print("1. 必須除外カテゴリーを除去")
    print("2. 高相関ペア（|r| ≥ 0.8）から片方を除去")
    print("3. VIF再計算し、VIF > 10 のカテゴリーを順次除去")
    print("4. 最終的にVIF < 5 を目標に調整")
    print("5. モデルの説明力（R²）との兼ね合いで微調整")
    
    # 4. 除外の優先順位
    if len(high_corr_pairs) > 0:
        print(f"\n【高相関ペア除外の優先順位】")
        print("以下のペアから片方ずつ除外を検討:")
        for i, (_, row) in enumerate(high_corr_pairs.head(5).iterrows()):
            print(f"  {i+1}. {row['Category1'][:30]}... ↔ {row['Category2'][:30]}... (r={row['Correlation']:.3f})")

def explain_vif_infinity():
    """
    VIF無限大の数学的・統計的説明
    """
    print("""
=== VIF無限大の詳細説明 ===

📊 **VIF（分散拡大係数）とは**
VIF = 1 / (1 - R²)
- R²: 当該変数を他の変数で回帰したときの決定係数
- R² → 1 のとき、VIF → ∞

🔍 **VIF無限大が発生する状況**

1. **完全相関 (Perfect Correlation)**
   - 数学的定義: r = ±1.0
   - 例: カテゴリーA = カテゴリーB
   - 原因: 同一概念の重複、階層関係

2. **線形従属 (Linear Dependence)**
   - 数学的定義: a×X₁ + b×X₂ + c×X₃ = 0
   - 例: Technology = Software + Hardware
   - 原因: カテゴリーの包含関係

3. **分散ゼロ (Zero Variance)**
   - 数学的定義: Var(X) = 0
   - 例: すべての企業で同じ値
   - 原因: データフィルタリング、ニッチカテゴリー

4. **数値的問題 (Numerical Issues)**
   - 行列の特異性（singular matrix）
   - 浮動小数点精度の限界
   - 条件数が非常に大きい行列

⚠️ **回帰分析への影響**
- 回帰係数が不安定になる
- 標準誤差が異常に大きくなる
- 統計的推定が信頼できなくなる
- 予測精度が低下する

💡 **対処法の優先順位**
1. 完全相関ペアの片方除去
2. 線形従属変数の除去
3. 分散ゼロ変数の除去
4. VIF > 10 の変数の除去
5. ビジネス重要度を考慮した最終調整

🎯 **目標**
- VIF < 5: 理想的
- VIF < 10: 許容範囲
- VIF > 10: 除外検討
- VIF = ∞: 必須除外
    """)

# 実行方法の説明
print("""
=== 改良版マルチコリニアリティ分析の実行方法 ===

【基本実行（VIF無限大対応版）】
1. 改良版デモ実行:
   results, mc_results = demo_multicollinearity_analysis_improved()

2. 特定期間での実行:
   results, mc_results = demo_multicollinearity_analysis_improved(specific_period=202403)

3. 閾値カスタマイズ:
   results, mc_results = demo_multicollinearity_analysis_improved(
       corr_threshold=0.8,
       vif_threshold=10.0
   )

【個別実行】
1. データ準備:
   results = main()

2. 改良版分析:
   mc_results = run_multicollinearity_analysis_improved(results)

3. VIF無限大の詳細説明:
   explain_vif_infinity()

【結果の活用】
- mc_results['vif_diagnosis']: VIF無限大の診断結果
- mc_results['summary']: 分析サマリー
- mc_results['vif_results']: 詳細VIF結果（問題分類付き）

【新機能】
✅ VIF無限大の自動検出・分類
✅ 完全相関の自動除去
✅ 線形従属の自動検出
✅ 問題別の色分け可視化
✅ 回帰分析戦略の自動生成
✅ 段階的変数選択の推奨

【VIF無限大の原因分類】
- Perfect_Correlation: 完全相関（赤色）
- Linear_Dependence: 線形従属（暗赤色）
- Zero_Variance: 分散ゼロ（灰色）
- Computational_Issue: 計算問題（オレンジ色）
""")

print("改良版マルチコリニアリティ分析が完成しました！")
